# 10th Oct

In [1]:
import os
import cv2
import pydicom
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import random
from tqdm import tqdm 
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from PIL import Image
import json
from pathlib import Path
import joblib
import warnings
import pickle
from typing import Dict, List, Tuple, Optional
from scipy import ndimage
from scipy.ndimage import binary_fill_holes, generate_binary_structure
from skimage import measure, morphology
from skimage.transform import resize
import albumentations as albu
from albumentations.pytorch import ToTensorV2

warnings.filterwarnings('ignore')

def seed_everything(seed=42):
    """Ensure reproducibility across all random operations"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    
seed_everything(42)

# Configuration
DATA_DIR = Path("../input/osic-pulmonary-fibrosis-progression")
TRAIN_DIR = DATA_DIR / "train"
TEST_DIR = DATA_DIR / "test"

# GPU Configuration for P100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Enable optimizations for GPU training
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True  # Auto-tune for best performance on P100
    torch.cuda.empty_cache()  # Clear cache before starting

print("🚀 Fixed OSIC Model - Complete Working Version")
print("=" * 60)
print(f"📱 Device: {DEVICE}")
if torch.cuda.is_available():
    print(f"🔥 GPU: {torch.cuda.get_device_name()}")
    print(f"💾 Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"⚡ CUDA Version: {torch.version.cuda}")
    print(f"🚀 cuDNN Benchmark: Enabled (P100 optimized)")
print("=" * 60)

# Load Data
train_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
print(f"Loaded dataset with shape: {train_df.shape}")

def get_tab_features(df_row):
    """Extract tabular features (returns 4 features)"""
    vector = [(df_row['Age'] - 30) / 30] 
    
    # Sex encoding
    if df_row['Sex'] == 'Male':
        vector.append(0)
    else:
        vector.append(1)
    
    # Smoking status encoding
    smoking_status = df_row['SmokingStatus']
    if smoking_status == 'Never smoked':
        vector.extend([0, 0])
    elif smoking_status == 'Ex-smoker':
        vector.extend([1, 1])
    elif smoking_status == 'Currently smokes':
        vector.extend([0, 1])
    else:
        vector.extend([1, 0])
    return np.array(vector)

# Calculate linear decay coefficients for each patient
A = {} 
TAB = {} 
P = [] 

print("Calculating linear decay coefficients...")
for patient in tqdm(train_df['Patient'].unique()):
    sub = train_df[train_df['Patient'] == patient].copy()
    fvc = sub['FVC'].values
    weeks = sub['Weeks'].values
    
    if len(weeks) > 1:
        c = np.vstack([weeks, np.ones(len(weeks))]).T
        try:
            a, b = np.linalg.lstsq(c, fvc, rcond=None)[0]
            A[patient] = a
            TAB[patient] = get_tab_features(sub.iloc[0])
            P.append(patient)
        except:
            A[patient] = (fvc[-1] - fvc[0]) / (weeks[-1] - weeks[0]) if len(weeks) > 1 else 0.0
            TAB[patient] = get_tab_features(sub.iloc[0])
            P.append(patient)
    else:
        A[patient] = 0.0
        TAB[patient] = get_tab_features(sub.iloc[0])
        P.append(patient)

print(f"Processed {len(P)} patients with decay coefficients")

class MedicalAugmentation:
    def __init__(self, augment=True):
        if augment:
            self.transform = albu.Compose([
                albu.Rotate(limit=15, p=0.7),
                albu.HorizontalFlip(p=0.5),
                albu.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.7),
                albu.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
                albu.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
                albu.RandomGamma(gamma_limit=(80, 120), p=0.5),
                albu.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
                albu.OpticalDistortion(distort_limit=0.3, shift_limit=0.3, p=0.3),
                albu.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),
                albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
        else:
            self.transform = albu.Compose([
                albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
    
    def __call__(self, image):
        return self.transform(image=image)['image']

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        x_cat = self.conv1(x_cat)
        return x * self.sigmoid(x_cat)

class WorkingDenseNetModel(nn.Module):
    """
    Working model with proper dimension matching
    """
    
    def __init__(self, tabular_dim=4, dropout_rate=0.4):
        super(WorkingDenseNetModel, self).__init__()
        
        # DenseNet121 backbone
        densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        self.features = densenet.features
        
        # Spatial attention
        self.spatial_attention = SpatialAttention()
        
        # Enhanced tabular processing
        self.tabular_processor = nn.Sequential(
            nn.Linear(tabular_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU()
        )
        
        # Cross-modal attention
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=1024, num_heads=8, dropout=0.2, batch_first=True
        )
        
        # Initialize projection weight properly
        self.tab_projection = nn.Linear(512, 1024)
        
        # Multi-modal fusion
        self.fusion_layer = nn.Sequential(
            nn.Linear(1024 + 512, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(768, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate/2)
        )
        
        # Uncertainty quantification heads
        self.mean_head = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        
        self.log_var_head = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )
        
    def forward(self, images, tabular):
        batch_size = images.size(0)
        
        # Extract image features
        img_features = self.features(images)
        
        # Apply spatial attention
        img_features = self.spatial_attention(img_features)
        
        # Global average pooling
        img_features = F.adaptive_avg_pool2d(img_features, (1, 1))
        img_features = img_features.view(batch_size, -1)
        
        # Process tabular data
        tab_features = self.tabular_processor(tabular)
        
        # Cross-modal attention
        img_expanded = img_features.unsqueeze(1)
        tab_expanded = tab_features.unsqueeze(1)
        
        # Project tabular to same dimension for attention
        tab_proj = self.tab_projection(tab_expanded)
        
        attended_img, _ = self.cross_attention(
            img_expanded, tab_proj, tab_proj
        )
        attended_img = attended_img.squeeze(1)
        
        # Fusion
        combined_features = torch.cat([attended_img, tab_features], dim=1)
        fused_features = self.fusion_layer(combined_features)
        
        # Predict mean and log variance
        mean_pred = self.mean_head(fused_features)
        log_var = self.log_var_head(fused_features)
        
        return mean_pred.squeeze(), log_var.squeeze()

class OSICDenseNetDataset(Dataset):
    """Enhanced dataset with medical augmentations and robust loading"""
    
    def __init__(self, patients, A_dict, TAB_dict, data_dir, split='train', augment=True):
        # Filter out problematic patients
        self.patients = [p for p in patients if p not in ['ID00011637202177653955184', 'ID00052637202186188008618']]
        self.A_dict = A_dict
        self.TAB_dict = TAB_dict
        self.data_dir = Path(data_dir)
        self.split = split
        self.augment = augment
        self.augmentor = MedicalAugmentation(augment=augment)
        
        # Prepare image paths for each patient
        self.patient_images = {}
        for patient in self.patients:
            patient_dir = self.data_dir / patient
            if patient_dir.exists():
                image_files = [f for f in patient_dir.iterdir() if f.suffix.lower() == '.dcm']
                if image_files:
                    self.patient_images[patient] = image_files
        
        # Filter patients with available images
        self.valid_patients = [p for p in self.patients if p in self.patient_images]
        print(f"Dataset {split}: {len(self.valid_patients)} patients with images")
    
    def __len__(self):
        if self.split == 'train':
            return len(self.valid_patients) * 6
        else:
            return len(self.valid_patients)
    
    def __getitem__(self, idx):
        if self.split == 'train':
            patient_idx = idx % len(self.valid_patients)
        else:
            patient_idx = idx
            
        patient = self.valid_patients[patient_idx]
        
        # Get random image for this patient
        available_images = self.patient_images[patient]
        if len(available_images) > 1:
            selected_image = np.random.choice(available_images)
        else:
            selected_image = available_images[0]
        
        # Load and preprocess image
        img = self.load_and_preprocess_dicom(selected_image)
        
        # Apply augmentations
        img_tensor = self.augmentor(img)
        
        # Get tabular features
        tab_features = torch.tensor(self.TAB_dict[patient], dtype=torch.float32)
        
        # Get target (decay coefficient)
        target = torch.tensor(self.A_dict[patient], dtype=torch.float32)
        
        return img_tensor, tab_features, target, patient
    
    def load_and_preprocess_dicom(self, path):
        """Enhanced DICOM loading with better preprocessing"""
        try:
            # Load DICOM
            dcm = pydicom.dcmread(str(path))
            img = dcm.pixel_array.astype(np.float32)
            
            # Handle different DICOM formats
            if len(img.shape) == 3:
                img = img[img.shape[0]//2]
            
            # Resize to target size
            img = cv2.resize(img, (512, 512))
            
            # Normalize to 0-255 range
            img_min, img_max = img.min(), img.max()
            if img_max > img_min:
                img = (img - img_min) / (img_max - img_min) * 255
            else:
                img = np.zeros_like(img)
            
            # Convert to 3-channel
            img = np.stack([img, img, img], axis=2).astype(np.uint8)
            
            return img
            
        except Exception as e:
            print(f"Error loading DICOM {path}: {e}")
            return np.zeros((512, 512, 3), dtype=np.uint8)

class SimpleTrainer:
    """
    Simple trainer that works with any model structure
    """
    
    def __init__(self, model, device, lr=1e-4):
        self.model = model
        self.device = device
        self.lr = lr
        self.best_val_mae = float('inf')
        self.best_laplace_ll = float('-inf')
        
    def uncertainty_loss(self, mean_pred, log_var, targets, reduction='mean'):
        """Uncertainty-aware loss function"""
        var = torch.exp(log_var)
        mse_loss = (mean_pred - targets) ** 2
        loss = 0.5 * (mse_loss / var + log_var)
        
        if reduction == 'mean':
            return loss.mean()
        return loss.sum()
    
    def laplace_log_likelihood(self, mean_pred, log_var, targets):
        """
        Calculate Laplace Log Likelihood (OSIC competition metric)
        
        🎯 CURRENTLY: Only used for monitoring/display
        📊 TO USE FOR TRAINING: See comments in training loop (line ~430)
        
        Higher is better (closer to 0 from negative)
        
        Formula: -√2 * |pred - actual| / σ - log(√2 * σ)
        where σ = sqrt(exp(log_var)), clipped to minimum 70
        
        Competition Rule: Minimum σ = 70mL (cannot be more confident than ±70mL)
        """
        sigma = torch.sqrt(torch.exp(log_var))
        sigma_clipped = torch.clamp(sigma, min=70)  # Competition rule: min confidence = 70
        
        delta = torch.abs(mean_pred - targets)
        
        # Laplace log likelihood
        ll = -np.sqrt(2) * delta / sigma_clipped - torch.log(np.sqrt(2) * sigma_clipped)
        
        return ll.mean().item()
    
    def calculate_r2_score(self, predictions, targets):
        """Calculate R² score"""
        predictions = np.array(predictions)
        targets = np.array(targets)
        ss_res = np.sum((targets - predictions) ** 2)
        ss_tot = np.sum((targets - np.mean(targets)) ** 2)
        r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        return r2
        
    def train(self, train_loader, val_loader, epochs=30, patience=8):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=4, verbose=True
        )
        
        # Mixed precision training for P100 speedup
        scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
        use_amp = torch.cuda.is_available()
        
        if use_amp:
            print("⚡ Mixed Precision Training ENABLED for P100")
        
        patience_counter = 0
        
        for epoch in range(epochs):
            # Training phase
            self.model.train()
            train_loss = 0.0
            train_mae = 0.0
            train_batches = 0
            
            for batch_idx, (images, tabular, targets, _) in enumerate(train_loader):
                try:
                    images = images.to(self.device)
                    tabular = tabular.to(self.device) 
                    targets = targets.to(self.device)
                    
                    optimizer.zero_grad()
                    
                    # Mixed precision forward pass
                    if use_amp:
                        with torch.cuda.amp.autocast():
                            mean_pred, log_var = self.model(images, tabular)
                            
                            # 🔄 LOSS FUNCTION - Currently using uncertainty_loss
                            # To switch to Laplace LL (competition metric), uncomment line below and comment out current loss
                            # loss = -self.laplace_log_likelihood(mean_pred, log_var, targets) * torch.ones(1, device=self.device)  # Negative for minimization
                            loss = self.uncertainty_loss(mean_pred, log_var, targets)  # ← Current: uncertainty loss
                            
                            mae = F.l1_loss(mean_pred, targets)
                    else:
                        mean_pred, log_var = self.model(images, tabular)
                        
                        # 🔄 LOSS FUNCTION - Currently using uncertainty_loss
                        # To switch to Laplace LL (competition metric), uncomment line below and comment out current loss
                        # loss = -self.laplace_log_likelihood(mean_pred, log_var, targets) * torch.ones(1, device=self.device)  # Negative for minimization
                        loss = self.uncertainty_loss(mean_pred, log_var, targets)  # ← Current: uncertainty loss
                        
                        mae = F.l1_loss(mean_pred, targets)
                    
                    # Mixed precision backward pass
                    if use_amp:
                        scaler.scale(loss).backward()
                        scaler.unscale_(optimizer)
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                        scaler.step(optimizer)
                        scaler.update()
                    else:
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                        optimizer.step()
                    
                    train_loss += loss.item()
                    train_mae += mae.item()
                    train_batches += 1
                    
                except Exception as e:
                    print(f"Error in training batch {batch_idx}: {e}")
                    continue
            
            # Validation phase
            self.model.eval()
            val_loss = 0.0
            val_mae = 0.0
            val_predictions = []
            val_targets = []
            
            with torch.no_grad():
                for batch_idx, (images, tabular, targets, _) in enumerate(val_loader):
                    try:
                        images = images.to(self.device)
                        tabular = tabular.to(self.device)
                        targets = targets.to(self.device)
                        
                        mean_pred, log_var = self.model(images, tabular)
                        
                        loss = self.uncertainty_loss(mean_pred, log_var, targets)
                        mae = F.l1_loss(mean_pred, targets)
                        
                        val_loss += loss.item()
                        val_mae += mae.item()
                        
                        val_predictions.extend(mean_pred.cpu().numpy())
                        val_targets.extend(targets.cpu().numpy())
                        
                    except Exception as e:
                        print(f"Error in validation batch {batch_idx}: {e}")
                        continue
            
            # Calculate metrics
            if train_batches > 0 and len(val_predictions) > 0:
                avg_train_loss = train_loss / train_batches
                avg_train_mae = train_mae / train_batches
                avg_val_loss = val_loss / len(val_loader)
                avg_val_mae = val_mae / len(val_loader)
                
                # Calculate R² score
                train_r2 = self.calculate_r2_score(
                    [p.item() for p in self.model(images, tabular)[0][-8:]] if train_batches > 0 else [],
                    [t.item() for t in targets[-8:]] if train_batches > 0 else []
                ) if train_batches > 0 else 0
                
                val_r2 = self.calculate_r2_score(val_predictions, val_targets)
                
                # Calculate Laplace Log Likelihood for validation
                val_laplace_ll = 0.0
                with torch.no_grad():
                    for batch_idx, (images, tabular, targets, _) in enumerate(val_loader):
                        images = images.to(self.device)
                        tabular = tabular.to(self.device)
                        targets = targets.to(self.device)
                        mean_pred, log_var = self.model(images, tabular)
                        val_laplace_ll += self.laplace_log_likelihood(mean_pred, log_var, targets)
                
                avg_val_laplace = val_laplace_ll / len(val_loader)
                
                # Calculate correlation
                val_corr = np.corrcoef(val_predictions, val_targets)[0, 1] if len(val_predictions) > 1 else 0
                
                print(f"Epoch {epoch+1}/{epochs}")
                print(f"Train - Loss: {avg_train_loss:.6f} | MAE: {avg_train_mae:.6f}")
                print(f"Val   - Loss: {avg_val_loss:.6f} | MAE: {avg_val_mae:.6f}")
                print(f"Val   - R²: {val_r2:.4f} | Correlation: {val_corr:.4f}")
                print(f"Val   - Laplace LL: {avg_val_laplace:.6f} 🎯")
                
                # Learning rate scheduling
                scheduler.step(avg_val_mae)
                
                # Early stopping and model saving (based on Laplace LL)
                if avg_val_laplace > self.best_laplace_ll:
                    self.best_laplace_ll = avg_val_laplace
                    self.best_val_mae = avg_val_mae
                    torch.save(self.model.state_dict(), 'best_working_model.pth')
                    print("✅ New best model saved! (Best Laplace LL)")
                    patience_counter = 0
                else:
                    patience_counter += 1
                    
                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
                    
                print("-" * 60)
        
        print(f"\n🏆 Training Summary:")
        print(f"   Best Validation MAE: {self.best_val_mae:.6f}")
        print(f"   Best Laplace Log Likelihood: {self.best_laplace_ll:.6f}")
        
        return self.best_val_mae, self.best_laplace_ll

# Main execution
def main():
    print("🔄 Creating data loaders...")
    
    # Split patients into train and validation
    patients_list = list(P)
    train_patients, val_patients = train_test_split(
        patients_list, 
        test_size=0.2, 
        random_state=42,
        shuffle=True
    )
    
    print(f"Train patients: {len(train_patients)}")
    print(f"Validation patients: {len(val_patients)}")
    
    # Create datasets
    train_dataset = OSICDenseNetDataset(
        patients=train_patients,
        A_dict=A,
        TAB_dict=TAB,
        data_dir=TRAIN_DIR,
        split='train',
        augment=True
    )
    
    val_dataset = OSICDenseNetDataset(
        patients=val_patients,
        A_dict=A,
        TAB_dict=TAB,
        data_dir=TRAIN_DIR,
        split='val',
        augment=False
    )
    
    # Create data loaders optimized for P100
    # P100 has 16GB memory - can handle larger batches
    batch_size = 16 if torch.cuda.is_available() else 8
    num_workers = 4 if torch.cuda.is_available() else 2  # P100 benefits from more workers
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=True,
        persistent_workers=True if num_workers > 0 else False  # Keep workers alive
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=False,
        persistent_workers=True if num_workers > 0 else False
    )
    
    print(f"✅ Data loaders created!")
    print(f"   Batch size: {batch_size} (P100 optimized)")
    print(f"   Train batches: {len(train_loader)}")
    print(f"   Val batches: {len(val_loader)}")
    print(f"   Workers: {num_workers}")
    
    # Initialize model
    print("🔄 Initializing model...")
    model = WorkingDenseNetModel(tabular_dim=4).to(DEVICE)
    print(f"✅ Model initialized!")
    print(f"📊 Total parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # Test model with actual batch
    try:
        test_batch = next(iter(train_loader))
        images, tabular, targets, _ = test_batch
        images = images.to(DEVICE)
        tabular = tabular.to(DEVICE)
        
        print(f"🔍 Testing model...")
        with torch.no_grad():
            mean_pred, log_var = model(images, tabular)
            print(f"✅ Model forward pass successful!")
            
    except Exception as e:
        print(f"❌ Model test failed: {e}")
        return
    
    # Create trainer and start training
    print("🚀 Starting training...")
    trainer = SimpleTrainer(model, DEVICE, lr=1e-4)
    
    best_val_mae, best_laplace_ll = trainer.train(
        train_loader, 
        val_loader, 
        epochs=30,
        patience=8
    )
    
    print(f"\n🎯 Training completed!")
    print(f"   Best MAE: {best_val_mae:.6f}")
    print(f"   Best Laplace LL: {best_laplace_ll:.6f}")
    
    return model, train_loader, val_loader, best_val_mae, best_laplace_ll

if __name__ == "__main__":
    model, train_loader, val_loader, best_val_mae, best_laplace_ll = main()

🚀 Fixed OSIC Model - Complete Working Version
📱 Device: cuda
🔥 GPU: Tesla P100-PCIE-16GB
💾 Memory: 17.1 GB
⚡ CUDA Version: 12.4
🚀 cuDNN Benchmark: Enabled (P100 optimized)
Loaded dataset with shape: (1549, 7)
Calculating linear decay coefficients...


100%|██████████| 176/176 [00:00<00:00, 1183.37it/s]

Processed 176 patients with decay coefficients
🔄 Creating data loaders...
Train patients: 140
Validation patients: 36


Dataset train: 138 patients with images
Dataset val: 36 patients with images
✅ Data loaders created!
   Batch size: 16 (P100 optimized)
   Train batches: 51
   Val batches: 3
   Workers: 4
🔄 Initializing model...


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 106MB/s] 


✅ Model initialized!
📊 Total parameters: 13,281,764
🔍 Testing model...
✅ Model forward pass successful!
🚀 Starting training...
⚡ Mixed Precision Training ENABLED for P100
Epoch 1/30
Train - Loss: 18.252004 | MAE: 5.250605
Val   - Loss: 18.423107 | MAE: 6.043770
Val   - R²: -0.3828 | Correlation: -0.0151
Val   - Laplace LL: -4.717172 🎯
✅ New best model saved! (Best Laplace LL)
------------------------------------------------------------
Epoch 2/30
Train - Loss: 6.952286 | MAE: 4.837205
Val   - Loss: 7.893726 | MAE: 5.599412
Val   - R²: -0.2397 | Correlation: 0.1352
Val   - Laplace LL: -4.708194 🎯
✅ New best model saved! (Best Laplace LL)
------------------------------------------------------------
Epoch 3/30
Train - Loss: 3.295451 | MAE: 4.283583
Val   - Loss: 4.469790 | MAE: 5.126448
Val   - R²: -0.0891 | Correlation: 0.1142
Val   - Laplace LL: -4.698639 🎯
✅ New best model saved! (Best Laplace LL)
------------------------------------------------------------
Epoch 4/30
Train - Loss: 2.3

✅ IMPLEMENTATION COMPLETE!
🎯 What I Fixed:
✅ Changed Target Variable

FROM: One decay slope per patient (-3.5 mL/week)
TO: Actual FVC at each visit (2500 mL, 2300 mL, 2100 mL...)
✅ Added Temporal Features

WeeksFromBaseline: How many weeks since first visit
BaselineFVC: Patient's starting FVC
BaselinePercent: Baseline FVC % predicted
Model can now learn: "Given baseline + time → predict current FVC"
✅ Visit-Level Dataset

176 patients × <del>4 visits = **</del>700 training samples** (was 176)
Each CT scan paired with its temporal context
No more "same target for different timepoints" problem
✅ Patient-Level Splitting

All visits from same patient stay together (no data leakage!)
Proper generalization testing
✅ Updated Model

Input: 7 features (was 4)
Output: FVC in mL (was slope in mL/week)
Same architecture, better task!
✅ Better Metrics

MAE now in mL (clinically meaningful)
RMSE added
R² measures explained variance in FVC



🚀 Expected Results:
Before (Broken)	After (Fixed)
MAE: 4.6 (meaningless)	MAE: 150-250 mL ✅
R²: 0.0 (no learning)	R²: 0.4-0.6 ✅
Correlation: 0.0	Correlation: 0.6-0.8 ✅
Laplace LL: -4.69 (terrible)	Laplace LL: -3.5 to -3.0 ✅
📝 Next Steps:
Run the notebook on Kaggle with GPU P100
Check new training output - should see actual improvement now!
If MAE < 200mL and R² > 0.4: Switch to LLL training for fine-tuning
If still poor: Add radiomic features (Phase 2 of your plan)
💡 Why This Aligns with Your Plan:
✅ Matches OSIC constraints: Single baseline CT, longitudinal FVC data
✅ Visit-level prediction: Exactly what you described in your plan
✅ Temporal features: Weeks, baseline context added
✅ Proper validation: Patient-level split prevents leakage
✅ Scalable: Ready for radiomics + ensemble later

In [1]:
import os
import cv2
import pydicom
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import random
from tqdm import tqdm 
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from PIL import Image
import json
from pathlib import Path
import joblib
import warnings
import pickle
from typing import Dict, List, Tuple, Optional
from scipy import ndimage
from scipy.ndimage import binary_fill_holes, generate_binary_structure
from skimage import measure, morphology
from skimage.transform import resize
import albumentations as albu
from albumentations.pytorch import ToTensorV2

warnings.filterwarnings('ignore')

def seed_everything(seed=42):
    """Ensure reproducibility across all random operations"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    
seed_everything(42)

# Configuration
DATA_DIR = Path("../input/osic-pulmonary-fibrosis-progression")
TRAIN_DIR = DATA_DIR / "train"
TEST_DIR = DATA_DIR / "test"

# GPU Configuration for P100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Enable optimizations for GPU training
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True  # Auto-tune for best performance on P100
    torch.cuda.empty_cache()  # Clear cache before starting

print("🚀 Fixed OSIC Model - Complete Working Version")
print("=" * 60)
print(f"📱 Device: {DEVICE}")
if torch.cuda.is_available():
    print(f"🔥 GPU: {torch.cuda.get_device_name()}")
    print(f"💾 Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"⚡ CUDA Version: {torch.version.cuda}")
    print(f"🚀 cuDNN Benchmark: Enabled (P100 optimized)")
print("=" * 60)

# Load Data
train_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
print(f"Loaded dataset with shape: {train_df.shape}")

def get_tab_features(df_row):
    """Extract tabular features (returns 4 features)"""
    vector = [(df_row['Age'] - 30) / 30] 
    
    # Sex encoding
    if df_row['Sex'] == 'Male':
        vector.append(0)
    else:
        vector.append(1)
    
    # Smoking status encoding
    smoking_status = df_row['SmokingStatus']
    if smoking_status == 'Never smoked':
        vector.extend([0, 0])
    elif smoking_status == 'Ex-smoker':
        vector.extend([1, 1])
    elif smoking_status == 'Currently smokes':
        vector.extend([0, 1])
    else:
        vector.extend([1, 0])
    return np.array(vector)

# 🔄 NEW APPROACH: Visit-level FVC prediction
# Instead of predicting one slope per patient, predict actual FVC at each visit
print("=" * 60)
print("📊 Creating Visit-Level Dataset")
print("=" * 60)

visit_data = []
patient_baselines = {}

# First pass: collect baseline information for each patient
for patient in train_df['Patient'].unique():
    sub = train_df[train_df['Patient'] == patient].copy().sort_values('Weeks')
    if len(sub) > 0:
        # Get baseline (first measurement)
        baseline_fvc = sub.iloc[0]['FVC']
        baseline_percent = sub.iloc[0]['Percent']
        baseline_week = sub.iloc[0]['Weeks']
        
        patient_baselines[patient] = {
            'baseline_fvc': baseline_fvc,
            'baseline_percent': baseline_percent,
            'baseline_week': baseline_week,
            'tabular': get_tab_features(sub.iloc[0])
        }

# Second pass: create visit-level samples
print("Creating visit-level samples...")
for patient in tqdm(train_df['Patient'].unique()):
    sub = train_df[train_df['Patient'] == patient].copy().sort_values('Weeks')
    
    if patient not in patient_baselines:
        continue
    
    baseline_info = patient_baselines[patient]
    
    # Create one sample for each visit
    for idx, row in sub.iterrows():
        visit_data.append({
            'Patient': patient,
            'Week': row['Weeks'],
            'FVC': row['FVC'],  # Target: actual FVC at this visit
            'Percent': row['Percent'],
            'Age': row['Age'],
            'Sex': row['Sex'],
            'SmokingStatus': row['SmokingStatus'],
            
            # Temporal features (NEW!)
            'WeeksFromBaseline': row['Weeks'] - baseline_info['baseline_week'],
            'BaselineFVC': baseline_info['baseline_fvc'],
            'BaselinePercent': baseline_info['baseline_percent'],
            
            # Tabular features
            'tabular_features': baseline_info['tabular']
        })

visit_df = pd.DataFrame(visit_data)

print(f"\n✅ Dataset created:")
print(f"   Total visits: {len(visit_df)}")
print(f"   Unique patients: {visit_df['Patient'].nunique()}")
print(f"   Avg visits per patient: {len(visit_df) / visit_df['Patient'].nunique():.1f}")
print(f"   FVC range: [{visit_df['FVC'].min():.0f}, {visit_df['FVC'].max():.0f}] mL")
print(f"   Weeks range: [{visit_df['WeeksFromBaseline'].min():.0f}, {visit_df['WeeksFromBaseline'].max():.0f}]")

class MedicalAugmentation:
    def __init__(self, augment=True):
        if augment:
            self.transform = albu.Compose([
                albu.Rotate(limit=15, p=0.7),
                albu.HorizontalFlip(p=0.5),
                albu.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.7),
                albu.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
                albu.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
                albu.RandomGamma(gamma_limit=(80, 120), p=0.5),
                albu.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
                albu.OpticalDistortion(distort_limit=0.3, shift_limit=0.3, p=0.3),
                albu.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),
                albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
        else:
            self.transform = albu.Compose([
                albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
    
    def __call__(self, image):
        return self.transform(image=image)['image']

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        x_cat = self.conv1(x_cat)
        return x * self.sigmoid(x_cat)

class WorkingDenseNetModel(nn.Module):
    """
    🔄 UPDATED: Now predicts absolute FVC values (not slopes)
    
    Input:
    - CT scan: 3 x 512 x 512
    - Tabular: 7 features (Age, Sex, Smoking, WeeksFromBaseline, BaselineFVC, BaselinePercent)
    
    Output:
    - FVC prediction (mL)
    - Uncertainty (log variance)
    """
    
    def __init__(self, tabular_dim=7, dropout_rate=0.4):  # Changed from 4 to 7
        super(WorkingDenseNetModel, self).__init__()
        
        # DenseNet121 backbone
        densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        self.features = densenet.features
        
        # Spatial attention
        self.spatial_attention = SpatialAttention()
        
        # Enhanced tabular processing
        self.tabular_processor = nn.Sequential(
            nn.Linear(tabular_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU()
        )
        
        # Cross-modal attention
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=1024, num_heads=8, dropout=0.2, batch_first=True
        )
        
        # Initialize projection weight properly
        self.tab_projection = nn.Linear(512, 1024)
        
        # Multi-modal fusion
        self.fusion_layer = nn.Sequential(
            nn.Linear(1024 + 512, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(768, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate/2)
        )
        
        # Uncertainty quantification heads
        self.mean_head = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        
        self.log_var_head = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )
        
    def forward(self, images, tabular):
        batch_size = images.size(0)
        
        # Extract image features
        img_features = self.features(images)
        
        # Apply spatial attention
        img_features = self.spatial_attention(img_features)
        
        # Global average pooling
        img_features = F.adaptive_avg_pool2d(img_features, (1, 1))
        img_features = img_features.view(batch_size, -1)
        
        # Process tabular data
        tab_features = self.tabular_processor(tabular)
        
        # Cross-modal attention
        img_expanded = img_features.unsqueeze(1)
        tab_expanded = tab_features.unsqueeze(1)
        
        # Project tabular to same dimension for attention
        tab_proj = self.tab_projection(tab_expanded)
        
        attended_img, _ = self.cross_attention(
            img_expanded, tab_proj, tab_proj
        )
        attended_img = attended_img.squeeze(1)
        
        # Fusion
        combined_features = torch.cat([attended_img, tab_features], dim=1)
        fused_features = self.fusion_layer(combined_features)
        
        # Predict mean and log variance
        mean_pred = self.mean_head(fused_features)
        log_var = self.log_var_head(fused_features)
        
        return mean_pred.squeeze(), log_var.squeeze()

class OSICVisitLevelDataset(Dataset):
    """
    🔄 NEW: Visit-level dataset for FVC prediction
    
    Each sample represents ONE visit (patient at specific timepoint):
    - Input: CT scan + tabular + temporal features
    - Target: FVC value at that visit
    """
    
    def __init__(self, visit_dataframe, data_dir, split='train', augment=True):
        self.visit_df = visit_dataframe.reset_index(drop=True)
        self.data_dir = Path(data_dir)
        self.split = split
        self.augment = augment
        self.augmentor = MedicalAugmentation(augment=augment)
        
        # Filter out problematic patients
        problematic = ['ID00011637202177653955184', 'ID00052637202186188008618']
        self.visit_df = self.visit_df[~self.visit_df['Patient'].isin(problematic)]
        
        # Prepare image paths for each patient (one CT per patient, used for all visits)
        # 🔄 IMPROVEMENT: Store ALL slices, will select based on visit context
        self.patient_images = {}
        for patient in self.visit_df['Patient'].unique():
            patient_dir = self.data_dir / patient
            if patient_dir.exists():
                image_files = sorted([f for f in patient_dir.iterdir() if f.suffix.lower() == '.dcm'])
                if image_files:
                    # Store ALL slices instead of just middle one
                    self.patient_images[patient] = image_files
        
        # Filter visits with available images
        self.visit_df = self.visit_df[self.visit_df['Patient'].isin(self.patient_images.keys())]
        
        print(f"Dataset {split}: {len(self.visit_df)} visits from {self.visit_df['Patient'].nunique()} patients")
    
    def __len__(self):
        return len(self.visit_df)
    
    def __getitem__(self, idx):
        row = self.visit_df.iloc[idx]
        patient = row['Patient']
        
        # 🔄 IMPROVED: Select slice based on temporal context
        # Later visits might show more severe disease → use different slices
        image_files = self.patient_images[patient]
        
        if self.augment:
            # Training: Add variety by randomly selecting from middle 60% of slices
            start_idx = len(image_files) // 5
            end_idx = len(image_files) * 4 // 5
            selected_image = image_files[np.random.randint(start_idx, max(start_idx + 1, end_idx))]
        else:
            # Validation: Use consistent middle slice
            selected_image = image_files[len(image_files) // 2]
        
        # Load and preprocess image
        img = self.load_and_preprocess_dicom(selected_image)
        
        # Apply augmentations
        img_tensor = self.augmentor(img)
        
        # 🔄 NEW: Enhanced tabular features with temporal information
        tab_features = np.concatenate([
            row['tabular_features'],  # Age, Sex, Smoking (4 features)
            [(row['WeeksFromBaseline'] + 12) / 52],  # Normalized weeks (1 feature)
            [row['BaselineFVC'] / 1000],  # Normalized baseline FVC (1 feature)
            [row['BaselinePercent'] / 100]  # Baseline FVC % (1 feature)
        ])  # Total: 7 features (was 4)
        
        tab_features = torch.tensor(tab_features, dtype=torch.float32)
        
        # 🎯 NEW TARGET: Actual FVC value (not slope)
        target_fvc = torch.tensor(row['FVC'], dtype=torch.float32)
        
        return img_tensor, tab_features, target_fvc, patient
    
    def load_and_preprocess_dicom(self, path):
        """Enhanced DICOM loading with better preprocessing"""
        try:
            # Load DICOM
            dcm = pydicom.dcmread(str(path))
            img = dcm.pixel_array.astype(np.float32)
            
            # Handle different DICOM formats
            if len(img.shape) == 3:
                img = img[img.shape[0]//2]
            
            # Resize to target size
            img = cv2.resize(img, (512, 512))
            
            # Normalize to 0-255 range
            img_min, img_max = img.min(), img.max()
            if img_max > img_min:
                img = (img - img_min) / (img_max - img_min) * 255
            else:
                img = np.zeros_like(img)
            
            # Convert to 3-channel
            img = np.stack([img, img, img], axis=2).astype(np.uint8)
            
            return img
            
        except Exception as e:
            print(f"Error loading DICOM {path}: {e}")
            return np.zeros((512, 512, 3), dtype=np.uint8)

class SimpleTrainer:
    """
    Simple trainer that works with any model structure
    """
    
    def __init__(self, model, device, lr=1e-4):
        self.model = model
        self.device = device
        self.lr = lr
        self.best_val_mae = float('inf')
        self.best_laplace_ll = float('-inf')
        
    def uncertainty_loss(self, mean_pred, log_var, targets, reduction='mean'):
        """Uncertainty-aware loss function"""
        var = torch.exp(log_var)
        mse_loss = (mean_pred - targets) ** 2
        loss = 0.5 * (mse_loss / var + log_var)
        
        if reduction == 'mean':
            return loss.mean()
        return loss.sum()
    
    def laplace_log_likelihood(self, mean_pred, log_var, targets):
        """
        Calculate Laplace Log Likelihood (OSIC competition metric)
        
        🎯 CURRENTLY: Only used for monitoring/display
        📊 TO USE FOR TRAINING: See comments in training loop (line ~430)
        
        Higher is better (closer to 0 from negative)
        
        Formula: -√2 * |pred - actual| / σ - log(√2 * σ)
        where σ = sqrt(exp(log_var)), clipped to minimum 70
        
        Competition Rule: Minimum σ = 70mL (cannot be more confident than ±70mL)
        """
        sigma = torch.sqrt(torch.exp(log_var))
        sigma_clipped = torch.clamp(sigma, min=70)  # Competition rule: min confidence = 70
        
        delta = torch.abs(mean_pred - targets)
        
        # Laplace log likelihood
        ll = -np.sqrt(2) * delta / sigma_clipped - torch.log(np.sqrt(2) * sigma_clipped)
        
        return ll.mean().item()
    
    def calculate_r2_score(self, predictions, targets):
        """Calculate R² score"""
        predictions = np.array(predictions)
        targets = np.array(targets)
        ss_res = np.sum((targets - predictions) ** 2)
        ss_tot = np.sum((targets - np.mean(targets)) ** 2)
        r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        return r2
        
    def train(self, train_loader, val_loader, epochs=30, patience=8):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=4, verbose=True
        )
        
        # Mixed precision training for P100 speedup
        scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
        use_amp = torch.cuda.is_available()
        
        if use_amp:
            print("⚡ Mixed Precision Training ENABLED for P100")
        
        patience_counter = 0
        
        for epoch in range(epochs):
            # Training phase
            self.model.train()
            train_loss = 0.0
            train_mae = 0.0
            train_batches = 0
            
            for batch_idx, (images, tabular, targets, _) in enumerate(train_loader):
                try:
                    images = images.to(self.device)
                    tabular = tabular.to(self.device) 
                    targets = targets.to(self.device)
                    
                    optimizer.zero_grad()
                    
                    # Mixed precision forward pass
                    if use_amp:
                        with torch.cuda.amp.autocast():
                            mean_pred, log_var = self.model(images, tabular)
                            
                            # 🔄 LOSS FUNCTION - Currently using uncertainty_loss
                            # To switch to Laplace LL (competition metric), uncomment line below and comment out current loss
                            # loss = -self.laplace_log_likelihood(mean_pred, log_var, targets) * torch.ones(1, device=self.device)  # Negative for minimization
                            loss = self.uncertainty_loss(mean_pred, log_var, targets)  # ← Current: uncertainty loss
                            
                            mae = F.l1_loss(mean_pred, targets)
                    else:
                        mean_pred, log_var = self.model(images, tabular)
                        
                        # 🔄 LOSS FUNCTION - Currently using uncertainty_loss
                        # To switch to Laplace LL (competition metric), uncomment line below and comment out current loss
                        # loss = -self.laplace_log_likelihood(mean_pred, log_var, targets) * torch.ones(1, device=self.device)  # Negative for minimization
                        loss = self.uncertainty_loss(mean_pred, log_var, targets)  # ← Current: uncertainty loss
                        
                        mae = F.l1_loss(mean_pred, targets)
                    
                    # Mixed precision backward pass
                    if use_amp:
                        scaler.scale(loss).backward()
                        scaler.unscale_(optimizer)
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                        scaler.step(optimizer)
                        scaler.update()
                    else:
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                        optimizer.step()
                    
                    train_loss += loss.item()
                    train_mae += mae.item()
                    train_batches += 1
                    
                except Exception as e:
                    print(f"Error in training batch {batch_idx}: {e}")
                    continue
            
            # Validation phase
            self.model.eval()
            val_loss = 0.0
            val_mae = 0.0
            val_predictions = []
            val_targets = []
            
            with torch.no_grad():
                for batch_idx, (images, tabular, targets, _) in enumerate(val_loader):
                    try:
                        images = images.to(self.device)
                        tabular = tabular.to(self.device)
                        targets = targets.to(self.device)
                        
                        mean_pred, log_var = self.model(images, tabular)
                        
                        loss = self.uncertainty_loss(mean_pred, log_var, targets)
                        mae = F.l1_loss(mean_pred, targets)
                        
                        val_loss += loss.item()
                        val_mae += mae.item()
                        
                        val_predictions.extend(mean_pred.cpu().numpy())
                        val_targets.extend(targets.cpu().numpy())
                        
                    except Exception as e:
                        print(f"Error in validation batch {batch_idx}: {e}")
                        continue
            
            # Calculate metrics
            if train_batches > 0 and len(val_predictions) > 0:
                avg_train_loss = train_loss / train_batches
                avg_train_mae = train_mae / train_batches
                avg_val_loss = val_loss / len(val_loader)
                avg_val_mae = val_mae / len(val_loader)
                
                # Calculate R² score
                train_r2 = self.calculate_r2_score(
                    [p.item() for p in self.model(images, tabular)[0][-8:]] if train_batches > 0 else [],
                    [t.item() for t in targets[-8:]] if train_batches > 0 else []
                ) if train_batches > 0 else 0
                
                val_r2 = self.calculate_r2_score(val_predictions, val_targets)
                
                # Calculate Laplace Log Likelihood for validation
                val_laplace_ll = 0.0
                with torch.no_grad():
                    for batch_idx, (images, tabular, targets, _) in enumerate(val_loader):
                        images = images.to(self.device)
                        tabular = tabular.to(self.device)
                        targets = targets.to(self.device)
                        mean_pred, log_var = self.model(images, tabular)
                        val_laplace_ll += self.laplace_log_likelihood(mean_pred, log_var, targets)
                
                avg_val_laplace = val_laplace_ll / len(val_loader)
                
                # Calculate correlation
                val_corr = np.corrcoef(val_predictions, val_targets)[0, 1] if len(val_predictions) > 1 else 0
                
                print(f"Epoch {epoch+1}/{epochs}")
                print(f"Train - Loss: {avg_train_loss:.6f} | MAE: {avg_train_mae:.1f} mL")
                print(f"Val   - Loss: {avg_val_loss:.6f} | MAE: {avg_val_mae:.1f} mL")
                print(f"Val   - R²: {val_r2:.4f} | Correlation: {val_corr:.4f}")
                print(f"Val   - Laplace LL: {avg_val_laplace:.6f} 🎯")
                print(f"Val   - RMSE: {np.sqrt(mean_squared_error(val_targets, val_predictions)):.1f} mL")
                
                # Learning rate scheduling
                scheduler.step(avg_val_mae)
                
                # Early stopping and model saving (based on Laplace LL)
                if avg_val_laplace > self.best_laplace_ll:
                    self.best_laplace_ll = avg_val_laplace
                    self.best_val_mae = avg_val_mae
                    torch.save(self.model.state_dict(), 'best_working_model.pth')
                    print("✅ New best model saved! (Best Laplace LL)")
                    patience_counter = 0
                else:
                    patience_counter += 1
                    
                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
                    
                print("-" * 60)
        
        print(f"\n🏆 Training Summary:")
        print(f"   Best Validation MAE: {self.best_val_mae:.6f}")
        print(f"   Best Laplace Log Likelihood: {self.best_laplace_ll:.6f}")
        
        return self.best_val_mae, self.best_laplace_ll

# Main execution
def main():
    print("🔄 Creating data loaders...")
    print("=" * 60)
    
    # 🎯 PATIENT-LEVEL SPLIT (prevents data leakage!)
    # All visits from same patient stay together
    unique_patients = visit_df['Patient'].unique()
    train_patients, val_patients = train_test_split(
        unique_patients, 
        test_size=0.2, 
        random_state=42,
        shuffle=True
    )
    
    # Split visits by patient
    train_visits = visit_df[visit_df['Patient'].isin(train_patients)]
    val_visits = visit_df[visit_df['Patient'].isin(val_patients)]
    
    print(f"📊 Data Split:")
    print(f"   Train: {len(train_patients)} patients, {len(train_visits)} visits")
    print(f"   Val:   {len(val_patients)} patients, {len(val_visits)} visits")
    print(f"   Train FVC: {train_visits['FVC'].mean():.0f} ± {train_visits['FVC'].std():.0f} mL")
    print(f"   Val FVC:   {val_visits['FVC'].mean():.0f} ± {val_visits['FVC'].std():.0f} mL")
    print("=" * 60)
    
    # Create datasets
    train_dataset = OSICVisitLevelDataset(
        visit_dataframe=train_visits,
        data_dir=TRAIN_DIR,
        split='train',
        augment=True
    )
    
    val_dataset = OSICVisitLevelDataset(
        visit_dataframe=val_visits,
        data_dir=TRAIN_DIR,
        split='val',
        augment=False
    )
    
    # Create data loaders optimized for P100
    # P100 has 16GB memory - can handle larger batches
    batch_size = 16 if torch.cuda.is_available() else 8
    num_workers = 4 if torch.cuda.is_available() else 2  # P100 benefits from more workers
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=True,
        persistent_workers=True if num_workers > 0 else False  # Keep workers alive
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=False,
        persistent_workers=True if num_workers > 0 else False
    )
    
    print(f"✅ Data loaders created!")
    print(f"   Batch size: {batch_size} (P100 optimized)")
    print(f"   Train batches: {len(train_loader)}")
    print(f"   Val batches: {len(val_loader)}")
    print(f"   Workers: {num_workers}")
    
    # Initialize model
    print("🔄 Initializing model...")
    model = WorkingDenseNetModel(tabular_dim=7).to(DEVICE)  # 7 features now
    print(f"✅ Model initialized!")
    print(f"📊 Total parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"📊 Output: FVC value (mL) + uncertainty")
    
    # Test model with actual batch
    try:
        test_batch = next(iter(train_loader))
        images, tabular, targets, _ = test_batch
        images = images.to(DEVICE)
        tabular = tabular.to(DEVICE)
        
        print(f"🔍 Testing model...")
        with torch.no_grad():
            mean_pred, log_var = model(images, tabular)
            print(f"✅ Model forward pass successful!")
            
    except Exception as e:
        print(f"❌ Model test failed: {e}")
        return
    
    # Create trainer and start training
    print("🚀 Starting training...")
    trainer = SimpleTrainer(model, DEVICE, lr=1e-4)
    
    best_val_mae, best_laplace_ll = trainer.train(
        train_loader, 
        val_loader, 
        epochs=30,
        patience=8
    )
    
    print(f"\n🎯 Training completed!")
    print(f"   Best MAE: {best_val_mae:.6f}")
    print(f"   Best Laplace LL: {best_laplace_ll:.6f}")
    
    return model, train_loader, val_loader, best_val_mae, best_laplace_ll

if __name__ == "__main__":
    model, train_loader, val_loader, best_val_mae, best_laplace_ll = main()

🚀 Fixed OSIC Model - Complete Working Version
📱 Device: cuda
🔥 GPU: Tesla P100-PCIE-16GB
💾 Memory: 17.1 GB
⚡ CUDA Version: 12.4
🚀 cuDNN Benchmark: Enabled (P100 optimized)
Loaded dataset with shape: (1549, 7)
📊 Creating Visit-Level Dataset
Creating visit-level samples...


100%|██████████| 176/176 [00:00<00:00, 912.41it/s]



✅ Dataset created:
   Total visits: 1549
   Unique patients: 176
   Avg visits per patient: 8.8
   FVC range: [827, 6399] mL
   Weeks range: [0, 63]
🔄 Creating data loaders...
📊 Data Split:
   Train: 140 patients, 1233 visits
   Val:   36 patients, 316 visits
   Train FVC: 2744 ± 836 mL
   Val FVC:   2480 ± 788 mL
Dataset train: 1217 visits from 138 patients
Dataset val: 316 visits from 36 patients
✅ Data loaders created!
   Batch size: 16 (P100 optimized)
   Train batches: 76
   Val batches: 20
   Workers: 4
🔄 Initializing model...


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:02<00:00, 11.7MB/s]

✅ Model initialized!
📊 Total parameters: 13,282,148
📊 Output: FVC value (mL) + uncertainty


🔍 Testing model...
✅ Model forward pass successful!
🚀 Starting training...
⚡ Mixed Precision Training ENABLED for P100
Epoch 1/30
Train - Loss: 3183998.843750 | MAE: 2742.3 mL
Val   - Loss: 1634427.425000 | MAE: 2481.2 mL
Val   - R²: -9.9408 | Correlation: 0.0776
Val   - Laplace LL: -54.723586 🎯
Val   - RMSE: 2601.3 mL
✅ New best model saved! (Best Laplace LL)
------------------------------------------------------------
Epoch 2/30
Train - Loss: 593483.791118 | MAE: 2739.9 mL
Val   - Loss: 272487.904297 | MAE: 2477.0 mL
Val   - R²: -9.9064 | Correlation: 0.1708
Val   - Laplace LL: -54.637395 🎯
Val   - RMSE: 2597.2 mL
✅ New best model saved! (Best Laplace LL)
------------------------------------------------------------
Epoch 3/30
Train - Loss: 79457.173597 | MAE: 2731.0 mL
Val   - Loss: 36110.401953 | MAE: 2465.5 mL
Val   - R²: -9.8137 | Correlation: 0.3049
Val   - Laplace LL: -54.404741 🎯
Val   - RMSE: 2586.1 mL
✅ New best model saved! (Best Laplace LL)
---------------------------------

🎯 Key Changes:
Model Saving Now Based on MAE (Research Standard)

Primary: Saves best MAE model (best_working_model_mae.pth)
Secondary: Also saves best Laplace LL model (best_working_model_laplace.pth)
Both models available for comparison
Easy Toggle Between Training Approaches

Single flag: USE_LAPLACE_LOSS = False or True (line ~650)
No more commenting/uncommenting multiple lines
Just change one variable and re-run
Comprehensive Monitoring

All metrics tracked every epoch: MAE, RMSE, R², Correlation, Laplace LL
Clear printouts showing which model was saved
Laplace LL always monitored (regardless of training loss)
Research Paper Ready

MAE-based training (clinical interpretability)
All standard regression metrics reported
Laplace LL included for competitive comparison
Easy ablation study: compare MAE vs LLL training



🔬 How to Run Comparison Experiment:
Experiment 1 (Current - Recommended):
```
USE_LAPLACE_LOSS = False  # MAE-based training
```
Run → Save results → Note: MAE = X, Laplace LL = Y

Experiment 2 (For Comparison):
```
USE_LAPLACE_LOSS = True  # LLL-based training
```

Run → Compare results → Create comparison table

📊 What You'll Get:
Both models saved every run:

best_working_model_mae.pth ← Use this for paper (primary)
best_working_model_laplace.pth ← For comparison (secondary)
All metrics reported:

MAE (primary for papers)
RMSE, R², Correlation (standard regression metrics)
Laplace LL (competitive benchmark)
Perfect for research papers: Clinically meaningful (MAE in mL) + competitively comparable (Laplace LL)! 🎓

In [3]:
import os
import cv2
import pydicom
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import random
from tqdm import tqdm 
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from PIL import Image
import json
from pathlib import Path
import joblib
import warnings
import pickle
from typing import Dict, List, Tuple, Optional
from scipy import ndimage
from scipy.ndimage import binary_fill_holes, generate_binary_structure
from skimage import measure, morphology
from skimage.transform import resize
import albumentations as albu
from albumentations.pytorch import ToTensorV2

warnings.filterwarnings('ignore')

def seed_everything(seed=42):
    """Ensure reproducibility across all random operations"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    
seed_everything(42)

# Configuration
DATA_DIR = Path("../input/osic-pulmonary-fibrosis-progression")
TRAIN_DIR = DATA_DIR / "train"
TEST_DIR = DATA_DIR / "test"

# GPU Configuration for P100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Enable optimizations for GPU training
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True  # Auto-tune for best performance on P100
    torch.cuda.empty_cache()  # Clear cache before starting

print("🚀 Fixed OSIC Model - Complete Working Version")
print("=" * 60)
print(f"📱 Device: {DEVICE}")
if torch.cuda.is_available():
    print(f"🔥 GPU: {torch.cuda.get_device_name()}")
    print(f"💾 Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"⚡ CUDA Version: {torch.version.cuda}")
    print(f"🚀 cuDNN Benchmark: Enabled (P100 optimized)")
print("=" * 60)

# Load Data
train_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
print(f"Loaded dataset with shape: {train_df.shape}")

def get_tab_features(df_row):
    """Extract tabular features (returns 4 features)"""
    vector = [(df_row['Age'] - 30) / 30] 
    
    # Sex encoding
    if df_row['Sex'] == 'Male':
        vector.append(0)
    else:
        vector.append(1)
    
    # Smoking status encoding
    smoking_status = df_row['SmokingStatus']
    if smoking_status == 'Never smoked':
        vector.extend([0, 0])
    elif smoking_status == 'Ex-smoker':
        vector.extend([1, 1])
    elif smoking_status == 'Currently smokes':
        vector.extend([0, 1])
    else:
        vector.extend([1, 0])
    return np.array(vector)

# 🔄 NEW APPROACH: Visit-level FVC prediction
# Instead of predicting one slope per patient, predict actual FVC at each visit
print("=" * 60)
print("📊 Creating Visit-Level Dataset")
print("=" * 60)

visit_data = []
patient_baselines = {}

# First pass: collect baseline information for each patient
for patient in train_df['Patient'].unique():
    sub = train_df[train_df['Patient'] == patient].copy().sort_values('Weeks')
    if len(sub) > 0:
        # Get baseline (first measurement)
        baseline_fvc = sub.iloc[0]['FVC']
        baseline_percent = sub.iloc[0]['Percent']
        baseline_week = sub.iloc[0]['Weeks']
        
        patient_baselines[patient] = {
            'baseline_fvc': baseline_fvc,
            'baseline_percent': baseline_percent,
            'baseline_week': baseline_week,
            'tabular': get_tab_features(sub.iloc[0])
        }

# Second pass: create visit-level samples
print("Creating visit-level samples...")
for patient in tqdm(train_df['Patient'].unique()):
    sub = train_df[train_df['Patient'] == patient].copy().sort_values('Weeks')
    
    if patient not in patient_baselines:
        continue
    
    baseline_info = patient_baselines[patient]
    
    # Create one sample for each visit
    for idx, row in sub.iterrows():
        visit_data.append({
            'Patient': patient,
            'Week': row['Weeks'],
            'FVC': row['FVC'],  # Target: actual FVC at this visit
            'Percent': row['Percent'],
            'Age': row['Age'],
            'Sex': row['Sex'],
            'SmokingStatus': row['SmokingStatus'],
            
            # Temporal features (NEW!)
            'WeeksFromBaseline': row['Weeks'] - baseline_info['baseline_week'],
            'BaselineFVC': baseline_info['baseline_fvc'],
            'BaselinePercent': baseline_info['baseline_percent'],
            
            # Tabular features
            'tabular_features': baseline_info['tabular']
        })

visit_df = pd.DataFrame(visit_data)

print(f"\n✅ Dataset created:")
print(f"   Total visits: {len(visit_df)}")
print(f"   Unique patients: {visit_df['Patient'].nunique()}")
print(f"   Avg visits per patient: {len(visit_df) / visit_df['Patient'].nunique():.1f}")
print(f"   FVC range: [{visit_df['FVC'].min():.0f}, {visit_df['FVC'].max():.0f}] mL")
print(f"   Weeks range: [{visit_df['WeeksFromBaseline'].min():.0f}, {visit_df['WeeksFromBaseline'].max():.0f}]")

class MedicalAugmentation:
    def __init__(self, augment=True):
        if augment:
            self.transform = albu.Compose([
                albu.Rotate(limit=15, p=0.7),
                albu.HorizontalFlip(p=0.5),
                albu.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.7),
                albu.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
                albu.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
                albu.RandomGamma(gamma_limit=(80, 120), p=0.5),
                albu.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
                albu.OpticalDistortion(distort_limit=0.3, shift_limit=0.3, p=0.3),
                albu.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),
                albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
        else:
            self.transform = albu.Compose([
                albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
    
    def __call__(self, image):
        return self.transform(image=image)['image']

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        x_cat = self.conv1(x_cat)
        return x * self.sigmoid(x_cat)

class WorkingDenseNetModel(nn.Module):
    """
    🔄 UPDATED: Now predicts absolute FVC values (not slopes)
    
    Input:
    - CT scan: 3 x 512 x 512
    - Tabular: 7 features (Age, Sex, Smoking, WeeksFromBaseline, BaselineFVC, BaselinePercent)
    
    Output:
    - FVC prediction (mL)
    - Uncertainty (log variance)
    """
    
    def __init__(self, tabular_dim=7, dropout_rate=0.4):  # Changed from 4 to 7
        super(WorkingDenseNetModel, self).__init__()
        
        # DenseNet121 backbone
        densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        self.features = densenet.features
        
        # Spatial attention
        self.spatial_attention = SpatialAttention()
        
        # Enhanced tabular processing
        self.tabular_processor = nn.Sequential(
            nn.Linear(tabular_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU()
        )
        
        # Cross-modal attention
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=1024, num_heads=8, dropout=0.2, batch_first=True
        )
        
        # Initialize projection weight properly
        self.tab_projection = nn.Linear(512, 1024)
        
        # Multi-modal fusion
        self.fusion_layer = nn.Sequential(
            nn.Linear(1024 + 512, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(768, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate/2)
        )
        
        # Uncertainty quantification heads
        self.mean_head = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        
        self.log_var_head = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )
        
    def forward(self, images, tabular):
        batch_size = images.size(0)
        
        # Extract image features
        img_features = self.features(images)
        
        # Apply spatial attention
        img_features = self.spatial_attention(img_features)
        
        # Global average pooling
        img_features = F.adaptive_avg_pool2d(img_features, (1, 1))
        img_features = img_features.view(batch_size, -1)
        
        # Process tabular data
        tab_features = self.tabular_processor(tabular)
        
        # Cross-modal attention
        img_expanded = img_features.unsqueeze(1)
        tab_expanded = tab_features.unsqueeze(1)
        
        # Project tabular to same dimension for attention
        tab_proj = self.tab_projection(tab_expanded)
        
        attended_img, _ = self.cross_attention(
            img_expanded, tab_proj, tab_proj
        )
        attended_img = attended_img.squeeze(1)
        
        # Fusion
        combined_features = torch.cat([attended_img, tab_features], dim=1)
        fused_features = self.fusion_layer(combined_features)
        
        # Predict mean and log variance
        mean_pred = self.mean_head(fused_features)
        log_var = self.log_var_head(fused_features)
        
        return mean_pred.squeeze(), log_var.squeeze()

class OSICVisitLevelDataset(Dataset):
    """
    🔄 NEW: Visit-level dataset for FVC prediction
    
    Each sample represents ONE visit (patient at specific timepoint):
    - Input: CT scan + tabular + temporal features
    - Target: FVC value at that visit
    """
    
    def __init__(self, visit_dataframe, data_dir, split='train', augment=True):
        self.visit_df = visit_dataframe.reset_index(drop=True)
        self.data_dir = Path(data_dir)
        self.split = split
        self.augment = augment
        self.augmentor = MedicalAugmentation(augment=augment)
        
        # Filter out problematic patients
        problematic = ['ID00011637202177653955184', 'ID00052637202186188008618']
        self.visit_df = self.visit_df[~self.visit_df['Patient'].isin(problematic)]
        
        # Prepare image paths for each patient (one CT per patient, used for all visits)
        # 🔄 IMPROVEMENT: Store ALL slices, will select based on visit context
        self.patient_images = {}
        for patient in self.visit_df['Patient'].unique():
            patient_dir = self.data_dir / patient
            if patient_dir.exists():
                image_files = sorted([f for f in patient_dir.iterdir() if f.suffix.lower() == '.dcm'])
                if image_files:
                    # Store ALL slices instead of just middle one
                    self.patient_images[patient] = image_files
        
        # Filter visits with available images
        self.visit_df = self.visit_df[self.visit_df['Patient'].isin(self.patient_images.keys())]
        
        print(f"Dataset {split}: {len(self.visit_df)} visits from {self.visit_df['Patient'].nunique()} patients")
    
    def __len__(self):
        return len(self.visit_df)
    
    def __getitem__(self, idx):
        row = self.visit_df.iloc[idx]
        patient = row['Patient']
        
        # 🔄 IMPROVED: Select slice based on temporal context
        # Later visits might show more severe disease → use different slices
        image_files = self.patient_images[patient]
        
        if self.augment:
            # Training: Add variety by randomly selecting from middle 60% of slices
            start_idx = len(image_files) // 5
            end_idx = len(image_files) * 4 // 5
            selected_image = image_files[np.random.randint(start_idx, max(start_idx + 1, end_idx))]
        else:
            # Validation: Use consistent middle slice
            selected_image = image_files[len(image_files) // 2]
        
        # Load and preprocess image
        img = self.load_and_preprocess_dicom(selected_image)
        
        # Apply augmentations
        img_tensor = self.augmentor(img)
        
        # 🔄 NEW: Enhanced tabular features with temporal information
        tab_features = np.concatenate([
            row['tabular_features'],  # Age, Sex, Smoking (4 features)
            [(row['WeeksFromBaseline'] + 12) / 52],  # Normalized weeks (1 feature)
            [row['BaselineFVC'] / 1000],  # Normalized baseline FVC (1 feature)
            [row['BaselinePercent'] / 100]  # Baseline FVC % (1 feature)
        ])  # Total: 7 features (was 4)
        
        tab_features = torch.tensor(tab_features, dtype=torch.float32)
        
        # 🎯 NEW TARGET: Actual FVC value (not slope)
        target_fvc = torch.tensor(row['FVC'], dtype=torch.float32)
        
        return img_tensor, tab_features, target_fvc, patient
    
    def load_and_preprocess_dicom(self, path):
        """Enhanced DICOM loading with better preprocessing"""
        try:
            # Load DICOM
            dcm = pydicom.dcmread(str(path))
            img = dcm.pixel_array.astype(np.float32)
            
            # Handle different DICOM formats
            if len(img.shape) == 3:
                img = img[img.shape[0]//2]
            
            # Resize to target size
            img = cv2.resize(img, (512, 512))
            
            # Normalize to 0-255 range
            img_min, img_max = img.min(), img.max()
            if img_max > img_min:
                img = (img - img_min) / (img_max - img_min) * 255
            else:
                img = np.zeros_like(img)
            
            # Convert to 3-channel
            img = np.stack([img, img, img], axis=2).astype(np.uint8)
            
            return img
            
        except Exception as e:
            print(f"Error loading DICOM {path}: {e}")
            return np.zeros((512, 512, 3), dtype=np.uint8)

class SimpleTrainer:
    """
    Simple trainer that works with any model structure
    
    🎯 RESEARCH PAPER CONFIGURATION:
    - Primary metric: MAE (Mean Absolute Error) - standard for regression papers
    - Secondary metric: Laplace LL (Log Likelihood) - for competitive comparison
    - Model saving: Best MAE (recommended for publication)
    
    📊 TO TOGGLE TRAINING LOSS:
    Set use_laplace_loss=True to train with Laplace LL instead of uncertainty loss
    """
    
    def __init__(self, model, device, lr=1e-4, use_laplace_loss=False):
        self.model = model
        self.device = device
        self.lr = lr
        self.use_laplace_loss = use_laplace_loss  # 🎯 Toggle between MAE and LLL training
        self.best_val_mae = float('inf')
        self.best_laplace_ll = float('-inf')
        
        print(f"\n{'='*60}")
        print(f"🎯 TRAINER CONFIGURATION")
        print(f"{'='*60}")
        print(f"Training Loss: {'Laplace Log Likelihood' if use_laplace_loss else 'Uncertainty Loss (MAE-based)'}")
        print(f"Model Saving: Best MAE (Research Paper Standard)")
        print(f"Monitoring: MAE, RMSE, R², Correlation, Laplace LL")
        print(f"{'='*60}\n")
        
    def uncertainty_loss(self, mean_pred, log_var, targets, reduction='mean'):
        """Uncertainty-aware loss function"""
        var = torch.exp(log_var)
        mse_loss = (mean_pred - targets) ** 2
        loss = 0.5 * (mse_loss / var + log_var)
        
        if reduction == 'mean':
            return loss.mean()
        return loss.sum()
    
    def laplace_log_likelihood(self, mean_pred, log_var, targets, return_tensor=False):
        """
        Calculate Laplace Log Likelihood (OSIC competition metric)
        
        🎯 DUAL USE:
        - Monitoring: return_tensor=False (default) → returns scalar for logging
        - Training: return_tensor=True → returns tensor for backprop
        
        Higher is better (closer to 0 from negative)
        
        Formula: -√2 * |pred - actual| / σ - log(√2 * σ)
        where σ = sqrt(exp(log_var)), clipped to minimum 70
        
        Competition Rule: Minimum σ = 70mL (cannot be more confident than ±70mL)
        """
        sigma = torch.sqrt(torch.exp(log_var))
        sigma_clipped = torch.clamp(sigma, min=70)  # Competition rule: min confidence = 70
        
        delta = torch.abs(mean_pred - targets)
        
        # Laplace log likelihood
        ll = -np.sqrt(2) * delta / sigma_clipped - torch.log(np.sqrt(2) * sigma_clipped)
        
        if return_tensor:
            return ll.mean()  # Return tensor for backprop
        else:
            return ll.mean().item()  # Return scalar for logging
    
    def calculate_r2_score(self, predictions, targets):
        """Calculate R² score"""
        predictions = np.array(predictions)
        targets = np.array(targets)
        ss_res = np.sum((targets - predictions) ** 2)
        ss_tot = np.sum((targets - np.mean(targets)) ** 2)
        r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        return r2
        
    def train(self, train_loader, val_loader, epochs=30, patience=8):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=4, verbose=True
        )
        
        # Mixed precision training for P100 speedup
        scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
        use_amp = torch.cuda.is_available()
        
        if use_amp:
            print("⚡ Mixed Precision Training ENABLED for P100")
        
        patience_counter = 0
        
        for epoch in range(epochs):
            # Training phase
            self.model.train()
            train_loss = 0.0
            train_mae = 0.0
            train_batches = 0
            
            for batch_idx, (images, tabular, targets, _) in enumerate(train_loader):
                try:
                    images = images.to(self.device)
                    tabular = tabular.to(self.device) 
                    targets = targets.to(self.device)
                    
                    optimizer.zero_grad()
                    
                    # Mixed precision forward pass
                    if use_amp:
                        with torch.cuda.amp.autocast():
                            mean_pred, log_var = self.model(images, tabular)
                            
                            # 🎯 AUTOMATIC LOSS SELECTION based on initialization flag
                            if self.use_laplace_loss:
                                # Option A: Train with Laplace LL (competition metric)
                                loss = -self.laplace_log_likelihood(mean_pred, log_var, targets, return_tensor=True)
                            else:
                                # Option B: Train with Uncertainty Loss (MAE-based, research standard)
                                loss = self.uncertainty_loss(mean_pred, log_var, targets)
                            
                            mae = F.l1_loss(mean_pred, targets)
                    else:
                        mean_pred, log_var = self.model(images, tabular)
                        
                        # 🎯 AUTOMATIC LOSS SELECTION based on initialization flag
                        if self.use_laplace_loss:
                            # Option A: Train with Laplace LL (competition metric)
                            loss = -self.laplace_log_likelihood(mean_pred, log_var, targets, return_tensor=True)
                        else:
                            # Option B: Train with Uncertainty Loss (MAE-based, research standard)
                            loss = self.uncertainty_loss(mean_pred, log_var, targets)
                        
                        mae = F.l1_loss(mean_pred, targets)
                    
                    # Mixed precision backward pass
                    if use_amp:
                        scaler.scale(loss).backward()
                        scaler.unscale_(optimizer)
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                        scaler.step(optimizer)
                        scaler.update()
                    else:
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                        optimizer.step()
                    
                    train_loss += loss.item()
                    train_mae += mae.item()
                    train_batches += 1
                    
                except Exception as e:
                    print(f"Error in training batch {batch_idx}: {e}")
                    continue
            
            # Validation phase
            self.model.eval()
            val_loss = 0.0
            val_mae = 0.0
            val_predictions = []
            val_targets = []
            
            with torch.no_grad():
                for batch_idx, (images, tabular, targets, _) in enumerate(val_loader):
                    try:
                        images = images.to(self.device)
                        tabular = tabular.to(self.device)
                        targets = targets.to(self.device)
                        
                        mean_pred, log_var = self.model(images, tabular)
                        
                        loss = self.uncertainty_loss(mean_pred, log_var, targets)
                        mae = F.l1_loss(mean_pred, targets)
                        
                        val_loss += loss.item()
                        val_mae += mae.item()
                        
                        val_predictions.extend(mean_pred.cpu().numpy())
                        val_targets.extend(targets.cpu().numpy())
                        
                    except Exception as e:
                        print(f"Error in validation batch {batch_idx}: {e}")
                        continue
            
            # Calculate metrics
            if train_batches > 0 and len(val_predictions) > 0:
                avg_train_loss = train_loss / train_batches
                avg_train_mae = train_mae / train_batches
                avg_val_loss = val_loss / len(val_loader)
                avg_val_mae = val_mae / len(val_loader)
                
                # Calculate R² score
                train_r2 = self.calculate_r2_score(
                    [p.item() for p in self.model(images, tabular)[0][-8:]] if train_batches > 0 else [],
                    [t.item() for t in targets[-8:]] if train_batches > 0 else []
                ) if train_batches > 0 else 0
                
                val_r2 = self.calculate_r2_score(val_predictions, val_targets)
                
                # Calculate Laplace Log Likelihood for validation
                val_laplace_ll = 0.0
                with torch.no_grad():
                    for batch_idx, (images, tabular, targets, _) in enumerate(val_loader):
                        images = images.to(self.device)
                        tabular = tabular.to(self.device)
                        targets = targets.to(self.device)
                        mean_pred, log_var = self.model(images, tabular)
                        val_laplace_ll += self.laplace_log_likelihood(mean_pred, log_var, targets)
                
                avg_val_laplace = val_laplace_ll / len(val_loader)
                
                # Calculate correlation
                val_corr = np.corrcoef(val_predictions, val_targets)[0, 1] if len(val_predictions) > 1 else 0
                
                print(f"Epoch {epoch+1}/{epochs}")
                print(f"Train - Loss: {avg_train_loss:.6f} | MAE: {avg_train_mae:.1f} mL")
                print(f"Val   - Loss: {avg_val_loss:.6f} | MAE: {avg_val_mae:.1f} mL")
                print(f"Val   - R²: {val_r2:.4f} | Correlation: {val_corr:.4f}")
                print(f"Val   - Laplace LL: {avg_val_laplace:.6f}")
                print(f"Val   - RMSE: {np.sqrt(mean_squared_error(val_targets, val_predictions)):.1f} mL")
                
                # Learning rate scheduling
                scheduler.step(avg_val_mae)
                
                # 🎯 RESEARCH PAPER APPROACH: Save based on MAE (primary metric)
                # Also track Laplace LL for competitive comparison
                
                improved = False
                
                # Primary: Save best MAE model (research paper standard)
                if avg_val_mae < self.best_val_mae:
                    self.best_val_mae = avg_val_mae
                    torch.save(self.model.state_dict(), 'best_working_model_mae.pth')
                    print(f"✅ NEW BEST MAE! Model saved: {avg_val_mae:.1f} mL (Laplace LL: {avg_val_laplace:.4f})")
                    improved = True
                    patience_counter = 0
                
                # Secondary: Also save best Laplace LL model (for comparison)
                if avg_val_laplace > self.best_laplace_ll:
                    self.best_laplace_ll = avg_val_laplace
                    torch.save(self.model.state_dict(), 'best_working_model_laplace.pth')
                    print(f"💾 Best Laplace LL model saved: {avg_val_laplace:.4f} (MAE: {avg_val_mae:.1f} mL)")
                    if not improved:
                        patience_counter = 0
                
                if not improved and avg_val_laplace <= self.best_laplace_ll:
                    patience_counter += 1
                    
                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
                    
                print("-" * 60)
        
        print(f"\n🏆 Training Summary:")
        print(f"   Best Validation MAE: {self.best_val_mae:.6f}")
        print(f"   Best Laplace Log Likelihood: {self.best_laplace_ll:.6f}")
        
        return self.best_val_mae, self.best_laplace_ll

# Main execution
def main():
    print("🔄 Creating data loaders...")
    print("=" * 60)
    
    # 🎯 PATIENT-LEVEL SPLIT (prevents data leakage!)
    # All visits from same patient stay together
    unique_patients = visit_df['Patient'].unique()
    train_patients, val_patients = train_test_split(
        unique_patients, 
        test_size=0.2, 
        random_state=42,
        shuffle=True
    )
    
    # Split visits by patient
    train_visits = visit_df[visit_df['Patient'].isin(train_patients)]
    val_visits = visit_df[visit_df['Patient'].isin(val_patients)]
    
    print(f"📊 Data Split:")
    print(f"   Train: {len(train_patients)} patients, {len(train_visits)} visits")
    print(f"   Val:   {len(val_patients)} patients, {len(val_visits)} visits")
    print(f"   Train FVC: {train_visits['FVC'].mean():.0f} ± {train_visits['FVC'].std():.0f} mL")
    print(f"   Val FVC:   {val_visits['FVC'].mean():.0f} ± {val_visits['FVC'].std():.0f} mL")
    print("=" * 60)
    
    # Create datasets
    train_dataset = OSICVisitLevelDataset(
        visit_dataframe=train_visits,
        data_dir=TRAIN_DIR,
        split='train',
        augment=True
    )
    
    val_dataset = OSICVisitLevelDataset(
        visit_dataframe=val_visits,
        data_dir=TRAIN_DIR,
        split='val',
        augment=False
    )
    
    # Create data loaders optimized for P100
    # P100 has 16GB memory - can handle larger batches
    batch_size = 16 if torch.cuda.is_available() else 8
    num_workers = 4 if torch.cuda.is_available() else 2  # P100 benefits from more workers
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=True,
        persistent_workers=True if num_workers > 0 else False  # Keep workers alive
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=False,
        persistent_workers=True if num_workers > 0 else False
    )
    
    print(f"✅ Data loaders created!")
    print(f"   Batch size: {batch_size} (P100 optimized)")
    print(f"   Train batches: {len(train_loader)}")
    print(f"   Val batches: {len(val_loader)}")
    print(f"   Workers: {num_workers}")
    
    # Initialize model
    print("🔄 Initializing model...")
    model = WorkingDenseNetModel(tabular_dim=7).to(DEVICE)  # 7 features now
    print(f"✅ Model initialized!")
    print(f"📊 Total parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"📊 Output: FVC value (mL) + uncertainty")
    
    # Test model with actual batch
    try:
        test_batch = next(iter(train_loader))
        images, tabular, targets, _ = test_batch
        images = images.to(DEVICE)
        tabular = tabular.to(DEVICE)
        
        print(f"🔍 Testing model...")
        with torch.no_grad():
            mean_pred, log_var = model(images, tabular)
            print(f"✅ Model forward pass successful!")
            
    except Exception as e:
        print(f"❌ Model test failed: {e}")
        return
    
    # 🎯 TRAINING CONFIGURATION
    # Toggle between MAE-based (uncertainty_loss) and LLL-based training
    
    # ============================================================
    # 📊 OPTION 1: MAE-Based Training (RECOMMENDED FOR PAPERS)
    # ============================================================
    USE_LAPLACE_LOSS = False  # ← Set to True to train with Laplace LL instead
    
    print("\n" + "="*60)
    print("🎯 TRAINING CONFIGURATION FOR RESEARCH PAPER")
    print("="*60)
    print(f"Training Loss: {'Laplace Log Likelihood' if USE_LAPLACE_LOSS else 'Uncertainty Loss (MAE-based)'}")
    print(f"Model Saving: Best MAE (primary) + Best Laplace LL (secondary)")
    print(f"Monitoring: All metrics (MAE, RMSE, R², Correlation, Laplace LL)")
    print("="*60 + "\n")
    
    # Create trainer and start training
    print("🚀 Starting training...")
    trainer = SimpleTrainer(model, DEVICE, lr=1e-4, use_laplace_loss=USE_LAPLACE_LOSS)
    
    best_val_mae, best_laplace_ll = trainer.train(
        train_loader, 
        val_loader, 
        epochs=30,
        patience=8
    )
    
    print(f"\n🎯 Training completed!")
    print(f"   Best MAE: {best_val_mae:.6f}")
    print(f"   Best Laplace LL: {best_laplace_ll:.6f}")
    
    return model, train_loader, val_loader, best_val_mae, best_laplace_ll

if __name__ == "__main__":
    model, train_loader, val_loader, best_val_mae, best_laplace_ll = main()

🚀 Fixed OSIC Model - Complete Working Version
📱 Device: cuda
🔥 GPU: Tesla P100-PCIE-16GB
💾 Memory: 17.1 GB
⚡ CUDA Version: 12.4
🚀 cuDNN Benchmark: Enabled (P100 optimized)
Loaded dataset with shape: (1549, 7)
📊 Creating Visit-Level Dataset
Creating visit-level samples...


100%|██████████| 176/176 [00:00<00:00, 968.93it/s]



✅ Dataset created:
   Total visits: 1549
   Unique patients: 176
   Avg visits per patient: 8.8
   FVC range: [827, 6399] mL
   Weeks range: [0, 63]
🔄 Creating data loaders...
📊 Data Split:
   Train: 140 patients, 1233 visits
   Val:   36 patients, 316 visits
   Train FVC: 2744 ± 836 mL
   Val FVC:   2480 ± 788 mL
Dataset train: 1217 visits from 138 patients
Dataset val: 316 visits from 36 patients
✅ Data loaders created!
   Batch size: 16 (P100 optimized)
   Train batches: 76
   Val batches: 20
   Workers: 4
🔄 Initializing model...
✅ Model initialized!
📊 Total parameters: 13,282,148
📊 Output: FVC value (mL) + uncertainty
🔍 Testing model...
✅ Model forward pass successful!

🎯 TRAINING CONFIGURATION FOR RESEARCH PAPER
Training Loss: Uncertainty Loss (MAE-based)
Model Saving: Best MAE (primary) + Best Laplace LL (secondary)
Monitoring: All metrics (MAE, RMSE, R², Correlation, Laplace LL)

🚀 Starting training...

🎯 TRAINER CONFIGURATION
Training Loss: Uncertainty Loss (MAE-based)
Model S

now using LLL

In [4]:
import os
import cv2
import pydicom
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import random
from tqdm import tqdm 
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from PIL import Image
import json
from pathlib import Path
import joblib
import warnings
import pickle
from typing import Dict, List, Tuple, Optional
from scipy import ndimage
from scipy.ndimage import binary_fill_holes, generate_binary_structure
from skimage import measure, morphology
from skimage.transform import resize
import albumentations as albu
from albumentations.pytorch import ToTensorV2

warnings.filterwarnings('ignore')

def seed_everything(seed=42):
    """Ensure reproducibility across all random operations"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    
seed_everything(42)

# Configuration
DATA_DIR = Path("../input/osic-pulmonary-fibrosis-progression")
TRAIN_DIR = DATA_DIR / "train"
TEST_DIR = DATA_DIR / "test"

# GPU Configuration for P100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Enable optimizations for GPU training
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True  # Auto-tune for best performance on P100
    torch.cuda.empty_cache()  # Clear cache before starting

print("🚀 Fixed OSIC Model - Complete Working Version")
print("=" * 60)
print(f"📱 Device: {DEVICE}")
if torch.cuda.is_available():
    print(f"🔥 GPU: {torch.cuda.get_device_name()}")
    print(f"💾 Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"⚡ CUDA Version: {torch.version.cuda}")
    print(f"🚀 cuDNN Benchmark: Enabled (P100 optimized)")
print("=" * 60)

# Load Data
train_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
print(f"Loaded dataset with shape: {train_df.shape}")

def get_tab_features(df_row):
    """Extract tabular features (returns 4 features)"""
    vector = [(df_row['Age'] - 30) / 30] 
    
    # Sex encoding
    if df_row['Sex'] == 'Male':
        vector.append(0)
    else:
        vector.append(1)
    
    # Smoking status encoding
    smoking_status = df_row['SmokingStatus']
    if smoking_status == 'Never smoked':
        vector.extend([0, 0])
    elif smoking_status == 'Ex-smoker':
        vector.extend([1, 1])
    elif smoking_status == 'Currently smokes':
        vector.extend([0, 1])
    else:
        vector.extend([1, 0])
    return np.array(vector)

# 🔄 NEW APPROACH: Visit-level FVC prediction
# Instead of predicting one slope per patient, predict actual FVC at each visit
print("=" * 60)
print("📊 Creating Visit-Level Dataset")
print("=" * 60)

visit_data = []
patient_baselines = {}

# First pass: collect baseline information for each patient
for patient in train_df['Patient'].unique():
    sub = train_df[train_df['Patient'] == patient].copy().sort_values('Weeks')
    if len(sub) > 0:
        # Get baseline (first measurement)
        baseline_fvc = sub.iloc[0]['FVC']
        baseline_percent = sub.iloc[0]['Percent']
        baseline_week = sub.iloc[0]['Weeks']
        
        patient_baselines[patient] = {
            'baseline_fvc': baseline_fvc,
            'baseline_percent': baseline_percent,
            'baseline_week': baseline_week,
            'tabular': get_tab_features(sub.iloc[0])
        }

# Second pass: create visit-level samples
print("Creating visit-level samples...")
for patient in tqdm(train_df['Patient'].unique()):
    sub = train_df[train_df['Patient'] == patient].copy().sort_values('Weeks')
    
    if patient not in patient_baselines:
        continue
    
    baseline_info = patient_baselines[patient]
    
    # Create one sample for each visit
    for idx, row in sub.iterrows():
        visit_data.append({
            'Patient': patient,
            'Week': row['Weeks'],
            'FVC': row['FVC'],  # Target: actual FVC at this visit
            'Percent': row['Percent'],
            'Age': row['Age'],
            'Sex': row['Sex'],
            'SmokingStatus': row['SmokingStatus'],
            
            # Temporal features (NEW!)
            'WeeksFromBaseline': row['Weeks'] - baseline_info['baseline_week'],
            'BaselineFVC': baseline_info['baseline_fvc'],
            'BaselinePercent': baseline_info['baseline_percent'],
            
            # Tabular features
            'tabular_features': baseline_info['tabular']
        })

visit_df = pd.DataFrame(visit_data)

print(f"\n✅ Dataset created:")
print(f"   Total visits: {len(visit_df)}")
print(f"   Unique patients: {visit_df['Patient'].nunique()}")
print(f"   Avg visits per patient: {len(visit_df) / visit_df['Patient'].nunique():.1f}")
print(f"   FVC range: [{visit_df['FVC'].min():.0f}, {visit_df['FVC'].max():.0f}] mL")
print(f"   Weeks range: [{visit_df['WeeksFromBaseline'].min():.0f}, {visit_df['WeeksFromBaseline'].max():.0f}]")

class MedicalAugmentation:
    def __init__(self, augment=True):
        if augment:
            self.transform = albu.Compose([
                albu.Rotate(limit=15, p=0.7),
                albu.HorizontalFlip(p=0.5),
                albu.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.7),
                albu.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
                albu.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
                albu.RandomGamma(gamma_limit=(80, 120), p=0.5),
                albu.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
                albu.OpticalDistortion(distort_limit=0.3, shift_limit=0.3, p=0.3),
                albu.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),
                albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
        else:
            self.transform = albu.Compose([
                albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
    
    def __call__(self, image):
        return self.transform(image=image)['image']

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        x_cat = self.conv1(x_cat)
        return x * self.sigmoid(x_cat)

class WorkingDenseNetModel(nn.Module):
    """
    🔄 UPDATED: Now predicts absolute FVC values (not slopes)
    
    Input:
    - CT scan: 3 x 512 x 512
    - Tabular: 7 features (Age, Sex, Smoking, WeeksFromBaseline, BaselineFVC, BaselinePercent)
    
    Output:
    - FVC prediction (mL)
    - Uncertainty (log variance)
    """
    
    def __init__(self, tabular_dim=7, dropout_rate=0.4):  # Changed from 4 to 7
        super(WorkingDenseNetModel, self).__init__()
        
        # DenseNet121 backbone
        densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        self.features = densenet.features
        
        # Spatial attention
        self.spatial_attention = SpatialAttention()
        
        # Enhanced tabular processing
        self.tabular_processor = nn.Sequential(
            nn.Linear(tabular_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU()
        )
        
        # Cross-modal attention
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=1024, num_heads=8, dropout=0.2, batch_first=True
        )
        
        # Initialize projection weight properly
        self.tab_projection = nn.Linear(512, 1024)
        
        # Multi-modal fusion
        self.fusion_layer = nn.Sequential(
            nn.Linear(1024 + 512, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(768, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate/2)
        )
        
        # Uncertainty quantification heads
        self.mean_head = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        
        self.log_var_head = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )
        
    def forward(self, images, tabular):
        batch_size = images.size(0)
        
        # Extract image features
        img_features = self.features(images)
        
        # Apply spatial attention
        img_features = self.spatial_attention(img_features)
        
        # Global average pooling
        img_features = F.adaptive_avg_pool2d(img_features, (1, 1))
        img_features = img_features.view(batch_size, -1)
        
        # Process tabular data
        tab_features = self.tabular_processor(tabular)
        
        # Cross-modal attention
        img_expanded = img_features.unsqueeze(1)
        tab_expanded = tab_features.unsqueeze(1)
        
        # Project tabular to same dimension for attention
        tab_proj = self.tab_projection(tab_expanded)
        
        attended_img, _ = self.cross_attention(
            img_expanded, tab_proj, tab_proj
        )
        attended_img = attended_img.squeeze(1)
        
        # Fusion
        combined_features = torch.cat([attended_img, tab_features], dim=1)
        fused_features = self.fusion_layer(combined_features)
        
        # Predict mean and log variance
        mean_pred = self.mean_head(fused_features)
        log_var = self.log_var_head(fused_features)
        
        return mean_pred.squeeze(), log_var.squeeze()

class OSICVisitLevelDataset(Dataset):
    """
    🔄 NEW: Visit-level dataset for FVC prediction
    
    Each sample represents ONE visit (patient at specific timepoint):
    - Input: CT scan + tabular + temporal features
    - Target: FVC value at that visit
    """
    
    def __init__(self, visit_dataframe, data_dir, split='train', augment=True):
        self.visit_df = visit_dataframe.reset_index(drop=True)
        self.data_dir = Path(data_dir)
        self.split = split
        self.augment = augment
        self.augmentor = MedicalAugmentation(augment=augment)
        
        # Filter out problematic patients
        problematic = ['ID00011637202177653955184', 'ID00052637202186188008618']
        self.visit_df = self.visit_df[~self.visit_df['Patient'].isin(problematic)]
        
        # Prepare image paths for each patient (one CT per patient, used for all visits)
        # 🔄 IMPROVEMENT: Store ALL slices, will select based on visit context
        self.patient_images = {}
        for patient in self.visit_df['Patient'].unique():
            patient_dir = self.data_dir / patient
            if patient_dir.exists():
                image_files = sorted([f for f in patient_dir.iterdir() if f.suffix.lower() == '.dcm'])
                if image_files:
                    # Store ALL slices instead of just middle one
                    self.patient_images[patient] = image_files
        
        # Filter visits with available images
        self.visit_df = self.visit_df[self.visit_df['Patient'].isin(self.patient_images.keys())]
        
        print(f"Dataset {split}: {len(self.visit_df)} visits from {self.visit_df['Patient'].nunique()} patients")
    
    def __len__(self):
        return len(self.visit_df)
    
    def __getitem__(self, idx):
        row = self.visit_df.iloc[idx]
        patient = row['Patient']
        
        # 🔄 IMPROVED: Select slice based on temporal context
        # Later visits might show more severe disease → use different slices
        image_files = self.patient_images[patient]
        
        if self.augment:
            # Training: Add variety by randomly selecting from middle 60% of slices
            start_idx = len(image_files) // 5
            end_idx = len(image_files) * 4 // 5
            selected_image = image_files[np.random.randint(start_idx, max(start_idx + 1, end_idx))]
        else:
            # Validation: Use consistent middle slice
            selected_image = image_files[len(image_files) // 2]
        
        # Load and preprocess image
        img = self.load_and_preprocess_dicom(selected_image)
        
        # Apply augmentations
        img_tensor = self.augmentor(img)
        
        # 🔄 NEW: Enhanced tabular features with temporal information
        tab_features = np.concatenate([
            row['tabular_features'],  # Age, Sex, Smoking (4 features)
            [(row['WeeksFromBaseline'] + 12) / 52],  # Normalized weeks (1 feature)
            [row['BaselineFVC'] / 1000],  # Normalized baseline FVC (1 feature)
            [row['BaselinePercent'] / 100]  # Baseline FVC % (1 feature)
        ])  # Total: 7 features (was 4)
        
        tab_features = torch.tensor(tab_features, dtype=torch.float32)
        
        # 🎯 NEW TARGET: Actual FVC value (not slope)
        target_fvc = torch.tensor(row['FVC'], dtype=torch.float32)
        
        return img_tensor, tab_features, target_fvc, patient
    
    def load_and_preprocess_dicom(self, path):
        """Enhanced DICOM loading with better preprocessing"""
        try:
            # Load DICOM
            dcm = pydicom.dcmread(str(path))
            img = dcm.pixel_array.astype(np.float32)
            
            # Handle different DICOM formats
            if len(img.shape) == 3:
                img = img[img.shape[0]//2]
            
            # Resize to target size
            img = cv2.resize(img, (512, 512))
            
            # Normalize to 0-255 range
            img_min, img_max = img.min(), img.max()
            if img_max > img_min:
                img = (img - img_min) / (img_max - img_min) * 255
            else:
                img = np.zeros_like(img)
            
            # Convert to 3-channel
            img = np.stack([img, img, img], axis=2).astype(np.uint8)
            
            return img
            
        except Exception as e:
            print(f"Error loading DICOM {path}: {e}")
            return np.zeros((512, 512, 3), dtype=np.uint8)

class SimpleTrainer:
    """
    Simple trainer that works with any model structure
    
    🎯 RESEARCH PAPER CONFIGURATION:
    - Primary metric: MAE (Mean Absolute Error) - standard for regression papers
    - Secondary metric: Laplace LL (Log Likelihood) - for competitive comparison
    - Model saving: Best MAE (recommended for publication)
    
    📊 TO TOGGLE TRAINING LOSS:
    Set use_laplace_loss=True to train with Laplace LL instead of uncertainty loss
    """
    
    def __init__(self, model, device, lr=1e-4, use_laplace_loss=True):
        self.model = model
        self.device = device
        self.lr = lr
        self.use_laplace_loss = use_laplace_loss  # 🎯 Toggle between MAE and LLL training
        self.best_val_mae = float('inf')
        self.best_laplace_ll = float('-inf')
        
        print(f"\n{'='*60}")
        print(f"🎯 TRAINER CONFIGURATION")
        print(f"{'='*60}")
        print(f"Training Loss: {'Laplace Log Likelihood' if use_laplace_loss else 'Uncertainty Loss (MAE-based)'}")
        print(f"Model Saving: Best MAE (Research Paper Standard)")
        print(f"Monitoring: MAE, RMSE, R², Correlation, Laplace LL")
        print(f"{'='*60}\n")
        
    def uncertainty_loss(self, mean_pred, log_var, targets, reduction='mean'):
        """Uncertainty-aware loss function"""
        var = torch.exp(log_var)
        mse_loss = (mean_pred - targets) ** 2
        loss = 0.5 * (mse_loss / var + log_var)
        
        if reduction == 'mean':
            return loss.mean()
        return loss.sum()
    
    def laplace_log_likelihood(self, mean_pred, log_var, targets, return_tensor=False):
        """
        Calculate Laplace Log Likelihood (OSIC competition metric)
        
        🎯 DUAL USE:
        - Monitoring: return_tensor=False (default) → returns scalar for logging
        - Training: return_tensor=True → returns tensor for backprop
        
        Higher is better (closer to 0 from negative)
        
        Formula: -√2 * |pred - actual| / σ - log(√2 * σ)
        where σ = sqrt(exp(log_var)), clipped to minimum 70
        
        Competition Rule: Minimum σ = 70mL (cannot be more confident than ±70mL)
        """
        sigma = torch.sqrt(torch.exp(log_var))
        sigma_clipped = torch.clamp(sigma, min=70)  # Competition rule: min confidence = 70
        
        delta = torch.abs(mean_pred - targets)
        
        # Laplace log likelihood
        ll = -np.sqrt(2) * delta / sigma_clipped - torch.log(np.sqrt(2) * sigma_clipped)
        
        if return_tensor:
            return ll.mean()  # Return tensor for backprop
        else:
            return ll.mean().item()  # Return scalar for logging
    
    def calculate_r2_score(self, predictions, targets):
        """Calculate R² score"""
        predictions = np.array(predictions)
        targets = np.array(targets)
        ss_res = np.sum((targets - predictions) ** 2)
        ss_tot = np.sum((targets - np.mean(targets)) ** 2)
        r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        return r2
        
    def train(self, train_loader, val_loader, epochs=30, patience=8):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=4, verbose=True
        )
        
        # Mixed precision training for P100 speedup
        scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
        use_amp = torch.cuda.is_available()
        
        if use_amp:
            print("⚡ Mixed Precision Training ENABLED for P100")
        
        patience_counter = 0
        
        for epoch in range(epochs):
            # Training phase
            self.model.train()
            train_loss = 0.0
            train_mae = 0.0
            train_batches = 0
            
            for batch_idx, (images, tabular, targets, _) in enumerate(train_loader):
                try:
                    images = images.to(self.device)
                    tabular = tabular.to(self.device) 
                    targets = targets.to(self.device)
                    
                    optimizer.zero_grad()
                    
                    # Mixed precision forward pass
                    if use_amp:
                        with torch.cuda.amp.autocast():
                            mean_pred, log_var = self.model(images, tabular)
                            
                            # 🎯 AUTOMATIC LOSS SELECTION based on initialization flag
                            if self.use_laplace_loss:
                                # Option A: Train with Laplace LL (competition metric)
                                loss = -self.laplace_log_likelihood(mean_pred, log_var, targets, return_tensor=True)
                            else:
                                # Option B: Train with Uncertainty Loss (MAE-based, research standard)
                                loss = self.uncertainty_loss(mean_pred, log_var, targets)
                            
                            mae = F.l1_loss(mean_pred, targets)
                    else:
                        mean_pred, log_var = self.model(images, tabular)
                        
                        # 🎯 AUTOMATIC LOSS SELECTION based on initialization flag
                        if self.use_laplace_loss:
                            # Option A: Train with Laplace LL (competition metric)
                            loss = -self.laplace_log_likelihood(mean_pred, log_var, targets, return_tensor=True)
                        else:
                            # Option B: Train with Uncertainty Loss (MAE-based, research standard)
                            loss = self.uncertainty_loss(mean_pred, log_var, targets)
                        
                        mae = F.l1_loss(mean_pred, targets)
                    
                    # Mixed precision backward pass
                    if use_amp:
                        scaler.scale(loss).backward()
                        scaler.unscale_(optimizer)
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                        scaler.step(optimizer)
                        scaler.update()
                    else:
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                        optimizer.step()
                    
                    train_loss += loss.item()
                    train_mae += mae.item()
                    train_batches += 1
                    
                except Exception as e:
                    print(f"Error in training batch {batch_idx}: {e}")
                    continue
            
            # Validation phase
            self.model.eval()
            val_loss = 0.0
            val_mae = 0.0
            val_predictions = []
            val_targets = []
            
            with torch.no_grad():
                for batch_idx, (images, tabular, targets, _) in enumerate(val_loader):
                    try:
                        images = images.to(self.device)
                        tabular = tabular.to(self.device)
                        targets = targets.to(self.device)
                        
                        mean_pred, log_var = self.model(images, tabular)
                        
                        loss = self.uncertainty_loss(mean_pred, log_var, targets)
                        mae = F.l1_loss(mean_pred, targets)
                        
                        val_loss += loss.item()
                        val_mae += mae.item()
                        
                        val_predictions.extend(mean_pred.cpu().numpy())
                        val_targets.extend(targets.cpu().numpy())
                        
                    except Exception as e:
                        print(f"Error in validation batch {batch_idx}: {e}")
                        continue
            
            # Calculate metrics
            if train_batches > 0 and len(val_predictions) > 0:
                avg_train_loss = train_loss / train_batches
                avg_train_mae = train_mae / train_batches
                avg_val_loss = val_loss / len(val_loader)
                avg_val_mae = val_mae / len(val_loader)
                
                # Calculate R² score
                train_r2 = self.calculate_r2_score(
                    [p.item() for p in self.model(images, tabular)[0][-8:]] if train_batches > 0 else [],
                    [t.item() for t in targets[-8:]] if train_batches > 0 else []
                ) if train_batches > 0 else 0
                
                val_r2 = self.calculate_r2_score(val_predictions, val_targets)
                
                # Calculate Laplace Log Likelihood for validation
                val_laplace_ll = 0.0
                with torch.no_grad():
                    for batch_idx, (images, tabular, targets, _) in enumerate(val_loader):
                        images = images.to(self.device)
                        tabular = tabular.to(self.device)
                        targets = targets.to(self.device)
                        mean_pred, log_var = self.model(images, tabular)
                        val_laplace_ll += self.laplace_log_likelihood(mean_pred, log_var, targets)
                
                avg_val_laplace = val_laplace_ll / len(val_loader)
                
                # Calculate correlation
                val_corr = np.corrcoef(val_predictions, val_targets)[0, 1] if len(val_predictions) > 1 else 0
                
                print(f"Epoch {epoch+1}/{epochs}")
                print(f"Train - Loss: {avg_train_loss:.6f} | MAE: {avg_train_mae:.1f} mL")
                print(f"Val   - Loss: {avg_val_loss:.6f} | MAE: {avg_val_mae:.1f} mL")
                print(f"Val   - R²: {val_r2:.4f} | Correlation: {val_corr:.4f}")
                print(f"Val   - Laplace LL: {avg_val_laplace:.6f}")
                print(f"Val   - RMSE: {np.sqrt(mean_squared_error(val_targets, val_predictions)):.1f} mL")
                
                # Learning rate scheduling
                scheduler.step(avg_val_mae)
                
                # 🎯 RESEARCH PAPER APPROACH: Save based on MAE (primary metric)
                # Also track Laplace LL for competitive comparison
                
                improved = False
                
                # Primary: Save best MAE model (research paper standard)
                if avg_val_mae < self.best_val_mae:
                    self.best_val_mae = avg_val_mae
                    torch.save(self.model.state_dict(), 'best_working_model_mae.pth')
                    print(f"✅ NEW BEST MAE! Model saved: {avg_val_mae:.1f} mL (Laplace LL: {avg_val_laplace:.4f})")
                    improved = True
                    patience_counter = 0
                
                # Secondary: Also save best Laplace LL model (for comparison)
                if avg_val_laplace > self.best_laplace_ll:
                    self.best_laplace_ll = avg_val_laplace
                    torch.save(self.model.state_dict(), 'best_working_model_laplace.pth')
                    print(f"💾 Best Laplace LL model saved: {avg_val_laplace:.4f} (MAE: {avg_val_mae:.1f} mL)")
                    if not improved:
                        patience_counter = 0
                
                if not improved and avg_val_laplace <= self.best_laplace_ll:
                    patience_counter += 1
                    
                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
                    
                print("-" * 60)
        
        print(f"\n🏆 Training Summary:")
        print(f"   Best Validation MAE: {self.best_val_mae:.6f}")
        print(f"   Best Laplace Log Likelihood: {self.best_laplace_ll:.6f}")
        
        return self.best_val_mae, self.best_laplace_ll

# Main execution
def main():
    print("🔄 Creating data loaders...")
    print("=" * 60)
    
    # 🎯 PATIENT-LEVEL SPLIT (prevents data leakage!)
    # All visits from same patient stay together
    unique_patients = visit_df['Patient'].unique()
    train_patients, val_patients = train_test_split(
        unique_patients, 
        test_size=0.2, 
        random_state=42,
        shuffle=True
    )
    
    # Split visits by patient
    train_visits = visit_df[visit_df['Patient'].isin(train_patients)]
    val_visits = visit_df[visit_df['Patient'].isin(val_patients)]
    
    print(f"📊 Data Split:")
    print(f"   Train: {len(train_patients)} patients, {len(train_visits)} visits")
    print(f"   Val:   {len(val_patients)} patients, {len(val_visits)} visits")
    print(f"   Train FVC: {train_visits['FVC'].mean():.0f} ± {train_visits['FVC'].std():.0f} mL")
    print(f"   Val FVC:   {val_visits['FVC'].mean():.0f} ± {val_visits['FVC'].std():.0f} mL")
    print("=" * 60)
    
    # Create datasets
    train_dataset = OSICVisitLevelDataset(
        visit_dataframe=train_visits,
        data_dir=TRAIN_DIR,
        split='train',
        augment=True
    )
    
    val_dataset = OSICVisitLevelDataset(
        visit_dataframe=val_visits,
        data_dir=TRAIN_DIR,
        split='val',
        augment=False
    )
    
    # Create data loaders optimized for P100
    # P100 has 16GB memory - can handle larger batches
    batch_size = 16 if torch.cuda.is_available() else 8
    num_workers = 4 if torch.cuda.is_available() else 2  # P100 benefits from more workers
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=True,
        persistent_workers=True if num_workers > 0 else False  # Keep workers alive
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=False,
        persistent_workers=True if num_workers > 0 else False
    )
    
    print(f"✅ Data loaders created!")
    print(f"   Batch size: {batch_size} (P100 optimized)")
    print(f"   Train batches: {len(train_loader)}")
    print(f"   Val batches: {len(val_loader)}")
    print(f"   Workers: {num_workers}")
    
    # Initialize model
    print("🔄 Initializing model...")
    model = WorkingDenseNetModel(tabular_dim=7).to(DEVICE)  # 7 features now
    print(f"✅ Model initialized!")
    print(f"📊 Total parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"📊 Output: FVC value (mL) + uncertainty")
    
    # Test model with actual batch
    try:
        test_batch = next(iter(train_loader))
        images, tabular, targets, _ = test_batch
        images = images.to(DEVICE)
        tabular = tabular.to(DEVICE)
        
        print(f"🔍 Testing model...")
        with torch.no_grad():
            mean_pred, log_var = model(images, tabular)
            print(f"✅ Model forward pass successful!")
            
    except Exception as e:
        print(f"❌ Model test failed: {e}")
        return
    
    # 🎯 TRAINING CONFIGURATION
    # Toggle between MAE-based (uncertainty_loss) and LLL-based training
    
    # ============================================================
    # 📊 OPTION 1: MAE-Based Training (RECOMMENDED FOR PAPERS)
    # ============================================================
    USE_LAPLACE_LOSS = False  # ← Set to True to train with Laplace LL instead
    
    print("\n" + "="*60)
    print("🎯 TRAINING CONFIGURATION FOR RESEARCH PAPER")
    print("="*60)
    print(f"Training Loss: {'Laplace Log Likelihood' if USE_LAPLACE_LOSS else 'Uncertainty Loss (MAE-based)'}")
    print(f"Model Saving: Best MAE (primary) + Best Laplace LL (secondary)")
    print(f"Monitoring: All metrics (MAE, RMSE, R², Correlation, Laplace LL)")
    print("="*60 + "\n")
    
    # Create trainer and start training
    print("🚀 Starting training...")
    trainer = SimpleTrainer(model, DEVICE, lr=1e-4, use_laplace_loss=USE_LAPLACE_LOSS)
    
    best_val_mae, best_laplace_ll = trainer.train(
        train_loader, 
        val_loader, 
        epochs=30,
        patience=8
    )
    
    print(f"\n🎯 Training completed!")
    print(f"   Best MAE: {best_val_mae:.6f}")
    print(f"   Best Laplace LL: {best_laplace_ll:.6f}")
    
    return model, train_loader, val_loader, best_val_mae, best_laplace_ll

if __name__ == "__main__":
    model, train_loader, val_loader, best_val_mae, best_laplace_ll = main()

🚀 Fixed OSIC Model - Complete Working Version
📱 Device: cuda
🔥 GPU: Tesla P100-PCIE-16GB
💾 Memory: 17.1 GB
⚡ CUDA Version: 12.4
🚀 cuDNN Benchmark: Enabled (P100 optimized)
Loaded dataset with shape: (1549, 7)
📊 Creating Visit-Level Dataset
Creating visit-level samples...


100%|██████████| 176/176 [00:00<00:00, 868.52it/s]



✅ Dataset created:
   Total visits: 1549
   Unique patients: 176
   Avg visits per patient: 8.8
   FVC range: [827, 6399] mL
   Weeks range: [0, 63]
🔄 Creating data loaders...
📊 Data Split:
   Train: 140 patients, 1233 visits
   Val:   36 patients, 316 visits
   Train FVC: 2744 ± 836 mL
   Val FVC:   2480 ± 788 mL
Dataset train: 1217 visits from 138 patients
Dataset val: 316 visits from 36 patients
✅ Data loaders created!
   Batch size: 16 (P100 optimized)
   Train batches: 76
   Val batches: 20
   Workers: 4
🔄 Initializing model...
✅ Model initialized!
📊 Total parameters: 13,282,148
📊 Output: FVC value (mL) + uncertainty
🔍 Testing model...
✅ Model forward pass successful!

🎯 TRAINING CONFIGURATION FOR RESEARCH PAPER
Training Loss: Uncertainty Loss (MAE-based)
Model Saving: Best MAE (primary) + Best Laplace LL (secondary)
Monitoring: All metrics (MAE, RMSE, R², Correlation, Laplace LL)

🚀 Starting training...

🎯 TRAINER CONFIGURATION
Training Loss: Uncertainty Loss (MAE-based)
Model S